In [73]:
from decimal import Decimal, getcontext
from fractions import Fraction
from math import ceil
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import json

# System Creation Functions

In [74]:
getcontext().prec = 155

# priceFeeds = pd.read_csv("BatBntDaiEnjLinkManaMaticMkrMlnOceanOmgRenRsrUsdcUsdtWbtcEth.csv")
PRICE_FEEDS_PATH: str = (
    "https://bancorml.s3.us-east-2.amazonaws.com/price_feeds.parquet"
)
priceFeeds = pd.read_parquet(PRICE_FEEDS_PATH)
priceFeeds.columns = [col.upper() for col in priceFeeds.columns]


def systemBuilder(timestamp, whitelistedTokens, users):
    """
    @Dev
    Creates the necessary objects to start the simulation.
    """
    users = {user : dict(zip(
    [f(tokenName) for tokenName in whitelistedTokens for f in (lambda tokenName: f"{tokenName}Balance", lambda tokenName: f"bn{tokenName}Balance")],
    [[Decimal("100000000000")] if i%2 == 0 else [Decimal("0")] for i in range(len(whitelistedTokens)*2)]
    )) for user in users}
    for user in users:
        users[user].update({"timestamp" : [timestamp]})
    nonUsers =  {
        "masterVault" : {
            "timestamp" : [timestamp],
            },
        "protocolWallet" : {
            "timestamp" : [timestamp],
            "bnbntBalance" : [Decimal("0")]
            },
        "erc20Contracts" : {
            "timestamp" : [timestamp],
            },
        "stakingLedger" : {
            "timestamp" : [timestamp],
            },
        "vortexLedger" : {
            "timestamp" : [timestamp],
            "bntBalance" : [Decimal("0")],
            "vbntBurned" : [Decimal("0")]
            },
        "globalProtocolSettings" : {
            "timestamp" : [timestamp],
            "networkFee" : [Decimal("0.20")],
            "withdrawalFee" : [Decimal("0.0025")],
            "bntMinLiquidity" : [Decimal("10000")],
            "cooldownTime" : [Decimal("0")]
            }
        }
    pools = {f"{tokenName}Pool" : {
            "timestamp" : [timestamp],
            "tradingEnabled" : [False],
            "bntTradingLiquidity" : [Decimal("0")],
            f"{tokenName}TradingLiquidity" : [Decimal("0")],
            "tradingFee" : [Decimal("0.005")],
            "bntFundingLimit" : [Decimal("0")], 
            "bntRemainingFunding" : [Decimal("0")],
            "bntFundingAmount" : [Decimal("0")],
            "externalProtectionVaultBalance" : [Decimal("0")],
            "spotRate" : [Decimal("0")],
            "emaRate" : [Decimal("0")],
            "emaCompressedNumerator" : [Decimal("0")],
            "emaCompressedDenominator" : [Decimal("0")],
            "emaDeviation" : [Decimal("0")],
            "emaLastUpdated" : [0]
            } for tokenName in whitelistedTokens if tokenName != "bnt"
        }
    pendingWithdrawals = {user : {} for user in users}
    pendingWithdrawals["idCounter"] = "0"
    for user in users:
        users[user]["vbntBalance"][-1] = Decimal("0")
    factions = [users, nonUsers]
    masterVault = {f"{tokenName}Balance" : [Decimal("0")] for tokenName in whitelistedTokens}
    stakingLedger = {f"{tokenName}Balance" : [Decimal("0")] for tokenName in whitelistedTokens}
    erc20Contracts = {f"bn{tokenName}Balance" : [Decimal("0")] for tokenName in whitelistedTokens}
    nonUsers.update(pools)
    nonUsers["masterVault"].update(masterVault)
    nonUsers["stakingLedger"].update(stakingLedger)
    nonUsers["erc20Contracts"].update(erc20Contracts)
    return(users, nonUsers, pendingWithdrawals, factions)

def fundSuperuser(whitelistedTokens, users):
    """
    @Dev
    During mainnet and tenderly fork recreations, gives the superuser an arbitrary quantity of all tokens and pool tokens. 
    """
    for tokenName in whitelistedTokens:
        users["superuser"][f"{tokenName}Balance"][-1] = 1000000000000
        users["superuser"][f"bn{tokenName}Balance"][-1] = 1000000000000
    return(None)

def convertEmaFromCompressed(emaCompressedNumerator, emaCompressedDenominator):
    """
    @Dev
    Takes tokenName and the nonUsers dictionary as inputs.
    Returns the ema as a decimal object, calculated from emaCompressedNumerator and emaCompressedDenominator.
    """
    decimalEma = emaCompressedNumerator/emaCompressedDenominator
    return(decimalEma)


def setPools(tokenWhitelist, poolStats, globalSettings): # for starting the system from a blockchain snapshot
    """
    @Dev
    Allows the simulator to start from a snapshot of the blockchain, or a tenderly fork, or similar.
    """
    poolStats = pd.read_csv(poolStats, index_col = 0)
    globalSettings = pd.read_csv(globalSettings, index_col = 0)
    tokenWhitelist = pd.read_csv(tokenWhitelist, index_col = 0)
    poolStats = poolStats.astype(str)
    globalSettings = globalSettings.astype(str)
    tokenWhitelist = tokenWhitelist.astype(str)
    timestamp = int(globalSettings["blockNumber"]["Global"])
    whitelistedTokens = list(tokenWhitelist.index)
    if "vbnt" not in whitelistedTokens:
        whitelistedTokens.append("vbnt")
    users = ["superuser"]
    users, nonUsers, pendingWithdrawals, factions = systemBuilder(timestamp, whitelistedTokens, users)
    for tokenName in whitelistedTokens:
        if tokenName != "vbnt" and tokenName != "bnt":
            nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] = Decimal(str(poolStats["bntTradingLiquidity"][tokenName]))/Decimal("10")**Decimal("18")
            nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] = Decimal(str(poolStats["tknTradingLiquidity"][tokenName]))/Decimal("10")**Decimal("18")
            nonUsers[f"{tokenName}Pool"]["tradingEnabled"][-1] = bool(poolStats["tradingEnabled"][tokenName])
            nonUsers[f"{tokenName}Pool"]["tradingFee"][-1] = Decimal(str(poolStats["tradingFeePPM"][tokenName]))/Decimal("1000000")
            nonUsers[f"{tokenName}Pool"]["bntFundingLimit"][-1] = Decimal(str(poolStats["bntFundingLimit"][tokenName]))/Decimal("10")**Decimal("18")
            nonUsers[f"{tokenName}Pool"]["bntRemainingFunding"][-1] = Decimal(str(poolStats["bntRemainingFunding"][tokenName]))/Decimal("10")**Decimal("18")
            nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1] = Decimal(str(poolStats["bntFundingAmount"][tokenName]))/Decimal("10")**Decimal("18")
            nonUsers[f"{tokenName}Pool"]["externalProtectionVaultBalance"][-1] = Decimal(str(poolStats["externalProtectionVaultTknBalance"][tokenName]))/Decimal("10")**Decimal("18")
            nonUsers[f"{tokenName}Pool"]["spotRate"][-1] = Decimal(str(poolStats["spotRate"][tokenName]))
            nonUsers["masterVault"][f"{tokenName}Balance"][-1] = Decimal(str(poolStats["masterVaultTknBalance"][tokenName]))/Decimal("10")**Decimal("18")
            nonUsers["stakingLedger"][f"{tokenName}Balance"][-1] = Decimal(str(poolStats["stakedBalance"][tokenName]))/Decimal("10")**Decimal("18")
            emaCompressedNumerator = Decimal(str(poolStats["emaCompressedNumerator"][tokenName]))
            emaCompressedDenominator = Decimal(str(poolStats["emaCompressedDenominator"][tokenName]))
            nonUsers[f"{tokenName}Pool"]["emaCompressedNumerator"][-1] = emaCompressedNumerator
            nonUsers[f"{tokenName}Pool"]["emaCompressedDenominator"][-1] = emaCompressedDenominator
            nonUsers[f"{tokenName}Pool"]["emaRate"][-1] = convertEmaFromCompressed(emaCompressedNumerator, emaCompressedDenominator)
            nonUsers["erc20Contracts"][f"bn{tokenName}Balance"][-1] = Decimal(str(poolStats["bnTknTotalSupply"][tokenName]))/Decimal("10")**Decimal("18")  
        else:
            pass
    nonUsers["masterVault"]["bntBalance"][-1] = Decimal(str(globalSettings["masterVaultBntBalance"]["Global"]))/Decimal("10")**Decimal("18")
    nonUsers["stakingLedger"]["bntBalance"][-1] = Decimal(str(globalSettings["bntPoolStakedAmount"]["Global"]))/Decimal("10")**Decimal("18")
    nonUsers["vortexLedger"]["bntBalance"][-1] = Decimal(str(globalSettings["vortexBntBalance"]["Global"]))/Decimal("10")**Decimal("18")
    nonUsers["protocolWallet"]["bnbntBalance"][-1] = Decimal(str(globalSettings["bntPoolBnBntBalance"]["Global"]))/Decimal("10")**Decimal("18")
    nonUsers["globalProtocolSettings"]["networkFee"][-1] = Decimal(str(globalSettings["networkFeePPM"]["Global"]))/Decimal("1000000")
    nonUsers["globalProtocolSettings"]["withdrawalFee"][-1] = Decimal(str(globalSettings["withdrawalFeePPM"]["Global"]))/Decimal("1000000")
    nonUsers["globalProtocolSettings"]["bntMinLiquidity"][-1] = Decimal(str(globalSettings["minLiquidityForTrading"]["Global"]))/Decimal("10")**Decimal("18")
    nonUsers["globalProtocolSettings"]["cooldownTime"][-1] = Decimal(str(globalSettings["lockDuration"]["Global"]))/Decimal("10")**Decimal("18")
    nonUsers["erc20Contracts"]["bnbntBalance"][-1] = Decimal(str(globalSettings["bnBntTotalSupply"]["Global"]))/Decimal("10")**Decimal("18")      
    fundSuperuser(whitelistedTokens, users)
    return(timestamp, users, nonUsers, pendingWithdrawals, factions, whitelistedTokens)

# Support Functions

In [75]:
getcontext().prec = 155

def getVbntPrice(bnbntRate, bntPrice):
    """
    @Dev
    Takes bnbntRate and bntPrice as inputs. 
    This function is called by getPrices. 
    For simulation purposes, both bootstrapping the vbnt pool, and arbitrage trading vbnt, is dependent on the bnbnt:bnt rate. 
    i.e. it does not make sense to use historical vbnt price data as a price feed. 
    Returns the vbnt price in usd.
    """
    vbntPrice = bntPrice/bnbntRate
    return(vbntPrice)

def getPrices(tokenName, timestamp, priceFeeds=priceFeeds):
    """
    @Dev
    Takes tokenName and timestamp as inputs. 
    Takes an optional third argument, priceFeeds, which is a csv file. 
    The priceFeed argument uses the default priceFeed set at system creation if none is passed. 
    Returns tokenPrice, bntPrice for the current timestamp.
    """
    bntPrice = Decimal(priceFeeds.at[timestamp, "bnt"])
    if tokenName == "vbnt":
        stakedBnt = nonUsers["stakingLedger"]["bntBalance"][-1]
        bnbntSupply = nonUsers["erc20Contracts"]["bnbntBalance"][-1]
        bnbntRate = getBnbntRate(stakedBnt, bnbntSupply)
        tokenPrice = getVbntPrice(bnbntRate, bntPrice)
    else:
        tokenPrice = Decimal(priceFeeds.at[timestamp, tokenName])
    return(tokenPrice, bntPrice)

def compressEma(emaRate):
    """
    @Dev
    Takes a Decimal ema as an input, and returns a fraction as two separate outputs: emaCompressedNumerator and emaCompressedDenominator.
    This process ensures that the ema can be stored as a maximum 112-bit number.
    This function is critical for testing purposes, and allows this simulation to emulate the protocol behavior on Ethereum.  
    """
    ema = Fraction(emaRate)
    scaled = ceil((max(ema.numerator, ema.denominator) / (2**112-1)))
    emaCompressedNumerator = int(ema.numerator/scaled)
    emaCompressedDenominator = int(ema.denominator/scaled)
    return (emaCompressedNumerator, emaCompressedDenominator) 

def updateCompressedEma(lastSpot, lastEmaCompressedNumerator, lastEmaCompressedDenominator):
    """
    @Dev
    Takes the previous spotRate, and the emaCompressedNumerator and emaCompressedDenominator as inputs.  
    The compressed newEma is calculated as a Decimal, then compressed. 
    Returns NewEmaCompressedNumerator and NewEmaCompressedDenominator as integers.
    """
    alpha = Decimal(0.2)
    newEma = alpha*lastSpot + (1 - alpha)*Decimal(lastEmaCompressedNumerator/lastEmaCompressedDenominator)
    NewEmaCompressedNumerator, NewEmaCompressedDenominator = compressEma(newEma)
    return(NewEmaCompressedNumerator, NewEmaCompressedDenominator)

def measureEmaDeviation(newEma, NewEmaCompressedNumerator, NewEmaCompressedDenominator):
    """
    @Dev
    Takes the accerate emaRate, and the emaCompressedNumerator and emaCompressedDenominator as inputs. 
    Returns the deviation between these values as emaRate/emaCompressedRate.
    """
    emaDeviation = newEma*Decimal(NewEmaCompressedDenominator)/Decimal(NewEmaCompressedNumerator)
    return(emaDeviation)
                        
def getBntknRate(stakedTkn, bntknSupply):
    """
    @Dev
    Determines the bntkn issuance rate for tkn deposits, based on the staking ledger and the current bntkn supply.  
    """
    if bntknSupply == 0 or stakedTkn == 0:
        bntknRate = Decimal("1")
    else: 
        bntknRate = bntknSupply/stakedTkn
    print(f"stakedTkn is {stakedTkn:.18f} and bntknSupply is {bntknSupply:.18f}")
    print(f"bntknrate is {bntknRate:.18f}")
    return(bntknRate)

def getBntknAmount(bntknRate, tknAmount):
    """
    @Dev
    Takes bntknRate and tknAmount as input. 
    Returns the product of bntknRate and tknAmount. 
    This function is used to determine the issuance of bntkn during deposits, and the required bntkn amount for a given tkn amount during withdrawal.  
    """
    bntknAmount = bntknRate*tknAmount
    print(f"bntknRate is {bntknRate:.18f} and tknAmount is {tknAmount:.18f}")
    print(f"bntknAmount is {bntknAmount:.18f}")
    return(bntknAmount)

def isPriceStable(spotRate, emaRate):
    """
    @Dev
    Measures the deviation between the spot- and ema bnt/tkn rates. 
    Returns True if the deviation is less than 1%.
    """
    withinTolerance = Decimal("0.99")*emaRate <= spotRate <= Decimal("1.01")*emaRate
    print(f"spotRate is {spotRate:.18f} and emaRate is {emaRate:.18f}")
    print(f"withinTolerance is {withinTolerance}")
    return(withinTolerance)

def getAverageTknTradingLiquidity(bntTradingLiquidity, emaRate):
    """
    @Dev
    Returns the tkn trading liquidity, adjusted by the ema. 
    """
    averageTknTradingLiquidity = bntTradingLiquidity/emaRate
    print(f"bntTradingLiquidity is {bntTradingLiquidity:.18f} and emaRate is {emaRate:.18f}")
    print(f"averageTknTradingLiquidity is {averageTknTradingLiquidity:.18f}")
    return(averageTknTradingLiquidity)

def getTknExcess(masterVaultTknBalance, tknTradingLiquidity):
    """
    @Dev
    NOTE: The arg tknTradingLiquidity is contentious; averageTknTradingLiquidity may be more appropriate.
    NOTE: This function is called by both poolDepthAdjustment and withdraw - and both pass the tknTradingLiquidity instead of the averageTknTradingLiquidity at present.
    Returns the difference between the masterVault tknBalance and the average averageTknTradingLiquidity.
    """
    tknExcess = masterVaultTknBalance - tknTradingLiquidity
    print(f"masterVaultTknBalance is {masterVaultTknBalance:.18f} and tknTradingLiquidity is {tknTradingLiquidity:.18f}")
    print(f"tknExcess is {tknExcess:.18f}")
    return(tknExcess)

def getTknExcessBntEquivalence(tknExcess, emaRate):
    """
    @Dev
    Returns the equivalent bnt value of the non-trading tkn balance of the masterVault.
    """
    tknExcessBntEquivalence = tknExcess*emaRate
    print(f"tknExcess is {tknExcess:.18f} and emaRate is {emaRate:.18f}")
    print(f"tknExcessBntEquivalence is {tknExcessBntEquivalence:.18f}")
    return(tknExcessBntEquivalence)

def getTknIncrease(bntRemainingFunding, emaRate):
    """
    @Dev
    Returns the change in the tkn trading liquidty during a deposit, if the pool is able to exhaust its remaining bnt funding. 
    """
    tknIncrease = bntRemainingFunding/emaRate
    print(f"bntRemainingFunding is {bntRemainingFunding:.18f} and emaRate is {emaRate:.18f}")
    print(f"tknIncrease is {tknIncrease:.18f}")
    return(tknIncrease)

def getModifedTknIncrease(bntTradingLiquidity, tknTradingLiquidity, bntIncrease, emaRate): # temporary
    """
    @Dev
    This function is used in place of getTknIncrease(), and was introduced during the Bancor 3 Beta.
    In the final version of the protocol, this function is no longer required. 
    """
    return((bntTradingLiquidity + bntIncrease)/emaRate - tknTradingLiquidity)

def getModifedBntIncrease(bntTradingLiquidity, tknTradingLiquidity, tknExcess, emaRate): # temporary
    """
    @Dev
    This function is used in place of getTknIncrease(), and was introduced during the Bancor 3 Beta.
    In the final version of the protocol, this function is no longer required. 
    """
    return((tknExcess + tknTradingLiquidity)*emaRate - bntTradingLiquidity)

def poolDepthAdjustment(tradingEnabled, spotRate, emaRate, bntTradingLiquidity, tknTradingLiquidity, masterVaultTknBalance, bntRemainingFunding):
    """
    @Dev
    Returns the quantities of BNT and TKN to add to the pool trading liquidity during a deposit.
    If the pool has trading disabled, or if the ema and spot rates deviate by more than 1%, returns nil. 
    """
    print("Adjusting the pool depth.")
    if  tradingEnabled == True and isPriceStable(spotRate, emaRate) == True and bntRemainingFunding > 0:
        averageTknTradingLiquidity = getAverageTknTradingLiquidity(bntTradingLiquidity, emaRate)
        tknExcess = getTknExcess(masterVaultTknBalance, tknTradingLiquidity) # Justify this later - should it be (b + c) - b or (b_average + c) - b
        tknExcessBntEquivalence = getTknExcessBntEquivalence(tknExcess, emaRate)
        if averageTknTradingLiquidity <= tknExcess and bntTradingLiquidity <= bntRemainingFunding:
            print("case1")
            bntIncrease = bntTradingLiquidity
            # tknIncrease = averageTknTradingLiquidity
            tknIncrease = getModifedTknIncrease(bntTradingLiquidity, tknTradingLiquidity, bntIncrease, emaRate)
        elif averageTknTradingLiquidity <= tknExcess and bntTradingLiquidity > bntRemainingFunding or averageTknTradingLiquidity > tknExcess and tknExcessBntEquivalence >= bntRemainingFunding:
            print("case2")
            bntIncrease = bntRemainingFunding
            # tknIncrease = getTknIncrease(bntRemainingFunding, emaRate)
            tknIncrease = getModifedTknIncrease(bntTradingLiquidity, tknTradingLiquidity, bntIncrease, emaRate)
        elif tknExcess < averageTknTradingLiquidity and bntTradingLiquidity <= bntRemainingFunding or averageTknTradingLiquidity > tknExcess and bntTradingLiquidity > bntRemainingFunding and tknExcessBntEquivalence < bntRemainingFunding:
            print("case3")
            # bntIncrease = tknExcessBntEquivalence
            bntIncrease = getModifedBntIncrease(bntTradingLiquidity, tknTradingLiquidity, tknExcess, emaRate)
            tknIncrease = tknExcess
    else:
        bntIncrease = Decimal(0)
        tknIncrease = Decimal(0)
    print(f"tradingEnabled = {tradingEnabled}")
    print(f"spotRate is {spotRate:.18f}, emaRate is {emaRate:.18f}")
    print(f"bntTradingLiquidity is {bntTradingLiquidity:.18f}, masterVaultTknBalance is {masterVaultTknBalance:.18f}")
    print(f"bntRemainingFunding is {bntRemainingFunding:.18f}")
    print(f"bntIncrease is {bntIncrease:.18f}, tknIncrease is {tknIncrease:.18f}")
    return(bntIncrease, tknIncrease)

def getBnbntRate(stakedBnt, bnbntSupply):
    """
    @Dev
    Determines the bnbnt issuance rate, used for bnbnt issuance for the protocol during tkn deposits. 
    Also used to determine the exchange between users and the protocol during bnt deposits. 
    Output is determined by the bnt balance of the staking ledger and the current bnbnt supply.  
    """
    if stakedBnt == 0 and bnbntSupply == 0:
        bnbntRate = Decimal(1)
    else:
        bnbntRate = bnbntSupply/stakedBnt
    print(f"stakedBnt is {stakedBnt:.18f} and bnbntSupply is {bnbntSupply:.18f}")
    print(f"bnbntRate is {bnbntRate:.18f}")
    return(bnbntRate)

def getBnbntAmount(bnbntRate, bntAmount):
    """
    @Dev
    Returns the quantity of bntkn to issue to the user during deposits.
    When called during withdrawal of tkn, it is the amount of bnbnt renounced by the protocol. 
    """
    bnbntAmount = bnbntRate*bntAmount
    print(f"bnbntRate is {bnbntRate:.18f} and bntAmount is {bntAmount:.18f}")
    print(f"bnbntAmount is {bnbntAmount:.18f}")
    return(bnbntAmount)

def twiceMinimumLiquidity(bntMinLiquidity):
    """
    @Dev
    Returns the bntMinLiquidity multiplied by 2.
    """
    bntBootstrapLiquidity = 2*bntMinLiquidity
    print(f"bntMinLiquidity is {bntMinLiquidity:.18f}")
    print(f"bntBootstrapLiquidity is {bntBootstrapLiquidity:.18f}")
    return(bntBootstrapLiquidity)

def getTknBootstrapLiquidity(bntBootstrapLiquidity, virtualRate):
    """
    @Dev
    Returns the bntBootstrapLiquidity divided by virtualRate.
    This is the tkn equivalnce of twice the bntMinLiquidity (i.e. the bntBootstrapLiquidity)
    """ 
    tknBootstrapLiquidity = bntBootstrapLiquidity/virtualRate
    print(f"bntBootstrapLiquidity is {bntBootstrapLiquidity:.18f} and virtualRate is {virtualRate:.18f}")
    print(f"tknBootstrapLiquidity is {tknBootstrapLiquidity:.18f}")
    return(tknBootstrapLiquidity)

def testMinimumLiquidity(vaultTknBalance, tknBootstrapLiquidity):
    """
    @Dev
    Returns True if the vaultTknBalance is at least the required tknBootsrapLiquidity
    """
    requirementsMet = vaultTknBalance >= tknBootstrapLiquidity
    print(f"vaultTknBalance is {vaultTknBalance:.18f} and tknBootstrapLiquidity is {tknBootstrapLiquidity:.18f}")
    print(f"requirementsMet is {requirementsMet}")
    return(requirementsMet)

def isTradingEnabled(tokenName):
    """
    @Dev
    Takes tokenName as input.
    Returns True if tradingEnabled.
    """
    print("Checking if trading is enabled on the pool")
    tradingEnabled = nonUsers[f"{tokenName}Pool"]["tradingEnabled"][-1]
    print(f"tradingEnabled = {tradingEnabled}")
    return(tradingEnabled)

def isEmaUpdateAllowed(emaLastUpdated):
    """
    @Dev
    Returns True if the moving average has not been updated on the existing block.
    The ema is only allowed once per block, per pool, to protect against price manipulation tactics. 
    """
    updateAllowed = timestamp != emaLastUpdated
    print(f"the last block the ema was updated on was {emaLastUpdated} and the current block is {timestamp}")
    print(f"updatedAllowed = {updateAllowed}")
    return(updateAllowed)

def updateEma(lastSpot, lastEma):
    """
    @Dev
    The ema update is part of the Bancor security system, and is allowed to update only once per bloack per pool. 
    Importantly, this function is called before a trade is performed, therefore the ema is a lagging average. 
    If allowed, the function reads the last available spotRate and emaRate, and returns the updated emaRate. 
    Else, the last ema rate is returned instead.
    """
    alpha = Decimal(0.2)
    newEma = alpha*lastSpot + (1 - alpha)*lastEma
    print(f"lastEma = {lastEma:.18f}")
    print(f"newEma = {newEma:.18f}")
    return(newEma)

def getChangedBntTradingLiquidity(a, b, d, e, x, direction):
    """
    @Dev
    Takes the trading liquidities of the pool, its fees, and the direction of the trade as inputs. 
    Trading direction is denominated as follows: 'tkn' == trade tkn for bnt, 'bnt' == trade bnt for tkn.
    This function takes into account the effect of the Bancor vortex, which can have non-intuitive effects on the bnt liquidity.
    Returns the new bntTradingliquidity remaining after the trade is processed. 
    """
    print('getChangedBntTradingLiquidity -> a, b, d, e, x, direction', a, b, d, e, x, direction)
    if direction == 'tkn':
        updatedBntLiquidity = a*(b + d*x*(1 - e))/(b + x)
    elif direction == 'bnt':
        updatedBntLiquidity = (a*(a + x) + d*(1 - e)*(a*x + x**2))/(a + d*x)
    print(f"bntTradingLiquidity is {a:.18f} and tknTradingLiquidity is {b:.18f}")
    print(f"tradingFee is {d:.18f} and networkFee is {e:.18f}")
    print(f"{x:.18f} {direction} is being traded for {'bnt' if direction == 'tkn' else 'tkn'}")
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f}")
    return(updatedBntLiquidity)

def getChangedTknTradingLiquidity(a, b, d, x, direction):
    """
    @Dev
    Takes the trading liquidities of the pool, its fees, and the direction of the trade as inputs. 
    Trading direction is denominated as follows: 'tkn' == trade tkn for bnt, 'bnt' == trade bnt for tkn.
    This function takes into account the effect of the Bancor vortex, which can have non-intuitive effects on the bnt liquidity.
    Returns the new tknTradingliquidity remaining after the trade is processed. 
    """
    if direction == 'tkn':
        updatedTknLiquidity = b + x
    elif direction == 'bnt':
        updatedTknLiquidity = b*(a + d*x)/(a + x)
    print(f"bntTradingLiquidity is {a:.18f} and tknTradingLiquidity is {b:.18f}")
    print(f"tradingFee is {d:.18f}")
    print(f"{x:.18f} {direction} is being traded for {'bnt' if direction == 'tkn' else 'tkn'}")
    print(f"updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    return(updatedTknLiquidity)

def getTargetAmount(a, b, d, x, direction):
    """
    @Dev
    Takes the trading liquidities of the pool, the swapFee only, and the direction of the trade as inputs. 
    Trading direction is denominated as follows: 'tkn' == trade tkn for bnt, 'bnt' == trade bnt for tkn.
    This function takes into account the effect of the Bancor vortex, which can have non-intuitive effects on the bnt liquidity.
    Returns the quantity of the target asset purchased to the user. 
    """
    if direction == 'tkn':
        targetSentToUser = a*x*(1 - d)/(b + x)
    elif direction == 'bnt':
        targetSentToUser = b*x*(1 - d)/(a + x)
    print(f"bntTradingLiquidity is {a:.18f} and tknTradingLiquidity is {b:.18f}")
    print(f"tradingFee is {d:.18f}")
    print(f"{x:.18f} {direction} is being traded for {targetSentToUser:.18f} {'bnt' if direction == 'tkn' else 'tkn'}")
    return(targetSentToUser)

def vortexCollection(a, b, d, e, x, direction):
    """
    @Dev
    Takes the trading liquidities of the pool, its fees, and the direction of the trade as inputs. 
    Trading direction is denominated as follows: 'tkn' == trade tkn for bnt, 'bnt' == trade bnt for tkn.
    This function returns the bnt collected by the Bancor vortex during trading.
    """
    if direction == 'tkn':
        bntSentToVortex = a*d*e*x/(b + x)
    elif direction == 'bnt':
        bntSentToVortex = d*e*x*(a + x)/(a + d*x)
    print(f"bntTradingLiquidity is {a:.18f} and tknTradingLiquidity is {b:.18f}")
    print(f"tradingFee is {d:.18f} and networkFee is {e:.18f}")
    print(f"{x:.18f} {direction} is being traded for {'bnt' if direction == 'tkn' else 'tkn'}")
    print(f"bntSentToVortex is {bntSentToVortex:.18f}")
    return(bntSentToVortex)

def swapFeeCollection(a, b, d, e, x, direction):
    """
    @Dev
    Takes the trading liquidities of the pool, its fees, and the direction of the trade as inputs. 
    Trading direction is denominated as follows: 'tkn' == trade tkn for bnt, 'bnt' == trade bnt for tkn.
    This function returns the swapFee collected by liquidity providers during trading, which is added to the stakingLedger.
    """
    if direction == 'tkn':
        tradeFee = a*d*x*(1 - e)/(b + x)
    elif direction == 'bnt':
        tradeFee = b*d*x*(1 - e)/(a + x)
    print(f"bntTradingLiquidity is {a:.18f} and tknTradingLiquidity is {b:.18f}")
    print(f"tradingFee is {d:.18f} and networkFee is {e:.18f}")
    print(f"{x:.18f} {direction} is being traded for {'bnt' if direction == 'tkn' else 'tkn'}")
    print(f"tradeFee given to stakers is {tradeFee:.18f}")
    return(tradeFee)

def tradeTknforBnt(tknAmount, tokenName):
    """
    @Dev
    Takes tknAmount, tokenName as inputs. 
    The trading liquidities, are updated according to the swap algorithm. 
    Two different fee quantities are collected, denominated in bnt, which are added to the stakingLedger and the vortexLedger. 
    Importantly, the swapFee is also used to increment the bntFundingAmount and the bntRemainingFunding.
    This function returns bntSentToUser.
    Depending on context, bntSentToUser can be the amount of bnt token to send to the user, or used as input to tradeBntForTkn in a second swap.
    """
    print(f"Trading {tknAmount:.18f} {tokenName} for bnt.")
    handleEma(tokenName)
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    tknTradingLiquidity = nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1]
    tradingFee = nonUsers[f"{tokenName}Pool"]["tradingFee"][-1]
    networkFee = nonUsers["globalProtocolSettings"]["networkFee"][-1]
    direction = "tkn"
    updatedBntLiquidity = getChangedBntTradingLiquidity(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, tknAmount, direction)
    updatedTknLiquidity = getChangedTknTradingLiquidity(bntTradingLiquidity, tknTradingLiquidity, tradingFee, tknAmount, direction)
    bntSentToUser = getTargetAmount(bntTradingLiquidity, tknTradingLiquidity, tradingFee, tknAmount, direction)
    tradeFee = swapFeeCollection(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, tknAmount, direction)
    bntCollectedByVortex = vortexCollection(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, tknAmount, direction)
    nonUsers["masterVault"]["bntBalance"][-1] -= bntSentToUser
    nonUsers["masterVault"][f"{tokenName}Balance"][-1] += tknAmount
    nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] = updatedBntLiquidity
    nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] = updatedTknLiquidity
    nonUsers["stakingLedger"]["bntBalance"][-1] += tradeFee
    nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1] += tradeFee
    nonUsers[f"{tokenName}Pool"]["bntRemainingFunding"][-1] -= tradeFee
    nonUsers["vortexLedger"]["bntBalance"][-1] += bntCollectedByVortex
    newSpotRate = getSpotRate(updatedBntLiquidity, updatedTknLiquidity)
    nonUsers[f"{tokenName}Pool"]["spotRate"][-1] = newSpotRate
    return(bntSentToUser)

def tradeBntForTkn(bntAmount, tokenName):
    """
    @Dev
    Takes bntAmount, tokenName as inputs. 
    The trading liquidities are updated according to the swap algorithm. 
    Two different fee quantities are collected, one denominated in tkn and the other in bnt. 
    The tkn fee is added to the stakingLedger and the bnt fee is added to the vortexLedger. 
    This function returns tknSentToUser.
    """
    print(f"Trading {bntAmount:.18f} bnt for {tokenName}")
    handleEma(tokenName)
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    tknTradingLiquidity = nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1]
    tradingFee = nonUsers[f"{tokenName}Pool"]["tradingFee"][-1]
    networkFee = nonUsers["globalProtocolSettings"]["networkFee"][-1]
    direction = "bnt"
    updatedBntLiquidity = getChangedBntTradingLiquidity(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, bntAmount, direction)
    updatedTknLiquidity = getChangedTknTradingLiquidity(bntTradingLiquidity, tknTradingLiquidity, tradingFee, bntAmount, direction)
    tknSentToUser = getTargetAmount(bntTradingLiquidity, tknTradingLiquidity, tradingFee, bntAmount, direction)
    tradeFee = swapFeeCollection(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, bntAmount, direction)
    bntCollectedByVortex = vortexCollection(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, bntAmount, direction)
    nonUsers["masterVault"]["bntBalance"][-1] += bntAmount
    nonUsers["masterVault"][f"{tokenName}Balance"][-1] -= tknSentToUser
    nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] = updatedBntLiquidity
    nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] = updatedTknLiquidity
    nonUsers["stakingLedger"][f"{tokenName}Balance"][-1] += tradeFee
    print('** tradeFee', tradeFee)
    nonUsers["vortexLedger"]["bntBalance"][-1] += bntCollectedByVortex
    newSpotRate = getSpotRate(updatedBntLiquidity, updatedTknLiquidity)
    nonUsers[f"{tokenName}Pool"]["spotRate"][-1] = newSpotRate
    return(tknSentToUser)

def getBntAmount(withdrawValue, withdrawalFee):
    """
    @Dev
    Takes withdrawValue, withdrawalFee as inputs. 
    Returns the quantity of bnt to issue to the user during bnt withdrawals.
    """
    bntAmount = withdrawValue*(1 - withdrawalFee)
    print(f"withdrawalFee is {withdrawalFee:.18f} and withdrawValue is {withdrawValue:.18f} bnt")
    print(f"bntAmount is {bntAmount:.18f}")
    return(bntAmount)

def getTknAmount(bntknRate, bntknAmount):
    """
    @Dev
    Returns the target quantity of tkn to pass into the withdrawal algorithm.
    """
    tknAmount = bntknAmount/bntknRate
    print(f"bntknRate is {bntknRate:.18f} and bntknAmount is {bntknAmount:.18f}")
    print(f"tknAmount is {tknAmount:.18f}")
    return(tknAmount)

def getSpotRate(updatedBntLiquidity, updatedTknLiquidity):
    """
    @Dev
    Returns the spotRate, the quotient of the bntTradingLiquidity and tknTradingLiquidity.
    If both inputs are zero, returns zero. 
    """
    if updatedBntLiquidity == 0 and updatedTknLiquidity == 0:
        spotRate = Decimal(0)
    else:
        spotRate = updatedBntLiquidity/updatedTknLiquidity
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f} and updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    print(f"spotRate is {spotRate:.18f}")
    return(spotRate)

def hmaxSurplus(averageTknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee): 
    """
    @Dev
    Returns the maximum number of withdrawn TKN that can currently be supported under the arbitrage method.
    """
    b = averageTknTradingLiquidity
    c = tknExcess
    e = tknStakedBalance
    m = tradingFee
    n = withdrawalFee
    hmax = b*e*(e*n + m*(b + c - e))/((1 - m)*(b + c - e)*(b + c - e*(1 - n)))
    print("Calculating hmaxSurplus.")
    print(f"averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}")
    print(f"tradingFee is {m:.18f}, withdrawalFee is {n:.18f}")
    print(f"hmax is {hmax:.18f}")
    return(hmax)

def arbitrageWithdrawalSurplus(bntTradingLiquidity, averageTknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee, tknWithdrawValue):
    """
    @Dev
    Calculates the withdrawal output variables when the protocol can arbitrage the pool.
    """
    a = bntTradingLiquidity
    b = averageTknTradingLiquidity
    c = tknExcess
    e = tknStakedBalance
    m = tradingFee
    n = withdrawalFee
    x = tknWithdrawValue
    updatedBntLiquidity = a*(b*e - m*(b*e + x*(b + c - e*(1 - n))))/((1 - m)*(b*e + x*(b + c - e*(1 - n))))
    bntRenounced = Decimal("0") 
    updatedTknLiquidity = (b*e + x*(b + c + e*(n - 1)))/e
    tknSentToUser = (x*(1 - n)) 
    bntSentToUser = Decimal("0") 
    print("Withdraw method is arbitrageWithdrawalSurplus.")
    print(f"bntTradingLiquidity is {a:.18f}, averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}")
    print(f"tradingFee is {m:.18f}, withdrawalFee is {n:.18f}")
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f}, bntRenounced is {bntRenounced:.18f}, updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    print(f"tknSentToUser is {tknSentToUser:.18f}, bntSentToUser is {bntSentToUser:.18f}")
    return(updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser)

def defaultWithdrawalSurplusCovered(bntTradingLiquidity, tknTradingLiquidity, withdrawalFee, tknWithdrawValue):
    """
    @Dev
    Calculates the withdrawal output variables when arbitrage is forbidden, and the excess TKN is sufficient to cover the entire withdrawal amount.
    """
    x = tknWithdrawValue
    n = withdrawalFee
    updatedBntLiquidity = bntTradingLiquidity
    bntRenounced = Decimal("0") 
    updatedTknLiquidity = tknTradingLiquidity 
    tknSentToUser = (x*(1 - n)) 
    bntSentToUser = Decimal("0") 
    print("Withdraw method is defaultWithdrawalSurplusCovered.")
    print(f"bntTradingLiquidity is {bntTradingLiquidity:.18f}, tknTradingLiquidity is {tknTradingLiquidity:.18f}, withdrawalFee is {n:.18f}, tknWithdrawValue is {x:.18f}")
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f}, bntRenounced is {bntRenounced:.18f}, updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    print(f"tknSentToUser is {tknSentToUser:.18f}, bntSentToUser is {bntSentToUser:.18f}")
    return(updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser)
        
def defaultWithdrawalSurplusExposed(bntTradingLiquidity, averageTknTradingLiquidity, tknExcess, withdrawalFee, tknWithdrawValue):
    """
    @Dev
    Calculates the withdrawal output variables when arbitrage is forbidden, and the TKN trading liquidity must be used to cover the withdrawal amount.
    """
    a = bntTradingLiquidity
    b = averageTknTradingLiquidity
    c = tknExcess
    n = withdrawalFee
    x = tknWithdrawValue
    updatedBntLiquidity = a*(b + c - x*(1 - n))/b
    bntRenounced = a*(x*(1 - n) - c)/b 
    updatedTknLiquidity = b + c - x*(1 - n) 
    tknSentToUser = x*(1 - n)
    bntSentToUser = Decimal("0") 
    print("Withdraw method is defaultWithdrawalSurplusExposed.")
    print(f"bntTradingLiquidity is {a:.18f}, averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, withdrawalFee is {n:.18f}, tknWithdrawValue is {x:.18f}")
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f}, bntRenounced is {bntRenounced:.18f}, updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    print(f"tknSentToUser is {tknSentToUser:.18f}, bntSentToUser is {bntSentToUser:.18f}")
    return(updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser)
          
def hmaxDeficit(averageTknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee): # maximum withdrawal amount allowed before the arbitrage value exceeds exit fee
    """
    @Dev
    Returns the maximum number of withdrawn TKN that can currently be supported under the arbitrage method.
    """
    b = averageTknTradingLiquidity
    c = tknExcess
    e = tknStakedBalance
    m = tradingFee
    n = withdrawalFee
    hmax = b*e*(e*n - m*(b + c - e*(1 - n)))/((1 - m)*(b + c - e)*(b + c - e*(1 - n)))
    print("Calculating hmaxDeficit.")
    print(f"averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}")
    print(f"tradingFee is {m:.18f}, withdrawalFee is {n:.18f}")
    print(f"hmax is {hmax:.18f}")
    return(hmax)

def arbitrageWithdrawalDeficit(bntTradingLiquidity, averageTknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee, tknWithdrawValue):
    """
    @Dev
    Calculates the withdrawal output variables when the protocol can arbitrage the pool.
    """
    a = bntTradingLiquidity
    b = averageTknTradingLiquidity
    c = tknExcess
    e = tknStakedBalance
    m = tradingFee
    n = withdrawalFee
    x = tknWithdrawValue
    updatedBntLiquidity = a*b*e/(b*e + x*(1 - m)*(b + c - e*(1 - n)))
    bntRenounced = Decimal("0") 
    updatedTknLiquidity = (b*e + x*(b + c - e*(1 - n)))/e
    tknSentToUser = x*(1 - n) 
    bntSentToUser = Decimal("0") 
    print("Withdraw method is arbitrageWithdrawalDeficit.")
    print(f"bntTradingLiquidity is {a:.18f}, averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}")
    print(f"tradingFee is {m:.18f}, withdrawalFee is {n:.18f}, tknWithdrawValue is {x:.18f}")
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f}, bntRenounced is {bntRenounced:.18f}, updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    print(f"tknSentToUser is {tknSentToUser:.18f}, bntSentToUser is {bntSentToUser:.18f}")
    return(updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser)

def defaultWithdrawalDeficitCovered(tknTradingLiquidity, bntTradingLiquidity, averageTknTradingLiquidity, tknExcess, tknStakedBalance, withdrawalFee, tknWithdrawValue):
    """
    @Dev
    Calculates the withdrawal output variables when arbitrage is forbidden, and the excess TKN is sufficient to cover the entire withdrawal amount.
    """
    a = bntTradingLiquidity
    b = averageTknTradingLiquidity
    c = tknExcess
    e = tknStakedBalance
    n = withdrawalFee
    x = tknWithdrawValue
    updatedBntLiquidity = a 
    bntRenounced = Decimal("0") 
    updatedTknLiquidity = tknTradingLiquidity 
    tknSentToUser = x*(1 - n)*(b + c)/e
    bntSentToUser = a*x*(1 - n)*(e - b - c)/(b*e)
    print("Withdraw method is defaultWithdrawalDeficitCovered.")
    print(f"bntTradingLiquidity is {a:.18f}, averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}")
    print(f"withdrawalFee is {n:.18f}, tknWithdrawValue is {x:.18f}")
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f}, bntRenounced is {bntRenounced:.18f}, updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    print(f"tknSentToUser is {tknSentToUser:.18f}, bntSentToUser is {bntSentToUser:.18f}")
    return(updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser)
    
def defaultWithdrawalDeficitExposed(bntTradingLiquidity, averageTknTradingLiquidity, tknExcess, tknStakedBalance, withdrawalFee, tknWithdrawValue):
    """
    @Dev
    Calculates the withdrawal output variables when arbitrage is forbidden, and the excess TKN is sufficient to cover the entire withdrawal amount.
    """
    a = bntTradingLiquidity 
    b = averageTknTradingLiquidity 
    c = tknExcess 
    e = tknStakedBalance 
    n = withdrawalFee
    x = tknWithdrawValue
    updatedBntLiquidity = a*(b + c)*(e - x*(1 - n))/(b*e)
    bntRenounced = a*(b*e - (b + c)*(e - x*(1 - n)))/(b*e)
    updatedTknLiquidity = (b + c)*(e - x*(1 - n))/e
    tknSentToUser = x*(1 - n)*(b + c)/e
    bntSentToUser = a*x*(1 - n)*(e - b - c)/(b*e)
    print("Withdraw method is defaultWithdrawalDeficitExposed.")
    print(f"bntTradingLiquidity is {a:.18f}, averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}")
    print(f"withdrawalFee is {n:.18f}, tknWithdrawValue is {x:.18f}")
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f}, bntRenounced is {bntRenounced:.18f}, updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    print(f"tknSentToUser is {tknSentToUser:.18f}, bntSentToUser is {bntSentToUser:.18f}")
    return(updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser)

def externalProtection(bntTradingLiquidity, averageTknTradingLiquidity, withdrawalFee, bntSentToUser, externalProtectionTknBalance, tknWithdrawValue, tknSentToUser, tradingEnabled): 
    """
    @Dev
    This replaces any BNT that would have been received by the user with TKN.
    """
    a = bntTradingLiquidity
    b = averageTknTradingLiquidity
    n = withdrawalFee
    T = bntSentToUser
    w = externalProtectionTknBalance
    x = tknWithdrawValue
    S = tknSentToUser
    print("Checking for support from the externalProtection policy.")
    print(f"bntTradingLiquidity is {a:.18f}, averageTknTradingLiquidity is {b:.18f}, withdrawalFee is {n:.18f}")
    print(f"bntSentToUser is {T:.18f}, externalProtectionTknBalance is {w:.18f}, tknWithdrawValue is {x:.18f}, tknSentToUser is {S:.18f}")
    if tradingEnabled == False:
        bntSentToUser = Decimal("0")
        print("Using externalProtection during and illiquid withdrawal.") 
        externalProtectionCompensation = min(w, x*(1 - n) - S)
    elif T and w:
        if T*b > w*a:
            print("Partial coverage by externalProtection") 
            bntSentToUser = (T*b - w*a)/b
            externalProtectionCompensation = w
        else:
            print("Complete coverage by externalProtection")
            bntSentToUser = Decimal("0")
            externalProtectionCompensation = T*b/a
    else:
        print("No coverage by externalProtection") 
        bntSentToUser = T
        externalProtectionCompensation = Decimal("0")
    print(f"Updated bntSentToUser is {bntSentToUser:.18f} and externalProtectionCompensation is {externalProtectionCompensation:.18f}")
    return(bntSentToUser, externalProtectionCompensation) 

def hlim(averageTknTradingLiquidity, tknExcess, tknStakedBalance): 
    """
    @Dev
    Returns the maximum number of withdrawn TKN that can currently be supported under the arbitrage method.
    """
    b = averageTknTradingLiquidity
    c = tknExcess
    e = tknStakedBalance
    hlim = c*e/(b + c)
    print("Calculating hlim.")
    print(f"averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}")
    print(f"hlim is {hlim:.18f}")
    return(hlim) 

def illiquidWithdrawal(bntTradingLiquidity, tknTradingLiquidity, tknExcess, tknStakedBalance, withdrawalFee, tknWithdrawValue):
    """
    @Dev
    Calculates the withdrawal output variables when trading is disabled.
    """
    c = tknExcess
    e = tknStakedBalance
    n = withdrawalFee
    x = tknWithdrawValue
    updatedBntLiquidity = bntTradingLiquidity
    bntRenounced = Decimal("0") 
    updatedTknLiquidity = tknTradingLiquidity 
    if c/(1 - n) > e: 
        tknSentToUser = x*(1 - n)
    else: 
        tknSentToUser = c*x*(1 - n)/e
    bntSentToUser = Decimal("0") 
    print("Withdraw method is illiquidWithdrawal.")
    print(f"tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}, withdrawalFee is {n:.18f}, tknWithdrawValue is {x:.18f}")
    print(f"bntSentToUser is {bntSentToUser:.18f}, tknSentToUser is {tknSentToUser:.18f}")
    return(updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser)

def processWithdrawal(tknTradingLiquidity, bntTradingLiquidity, averageTknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee, externalProtectionTknBalance, tknWithdrawValue, tradingEnabled):
    """
    @Dev
    Evaluates the conditions, and calls the appropriate functions
    """
    # b = averageTknTradingLiquidity
    b = tknTradingLiquidity
    c = tknExcess
    e = tknStakedBalance
    n = withdrawalFee
    x = tknWithdrawValue
    print("Processing the withdrawal.")
    if tradingEnabled == False: 
        updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser = illiquidWithdrawal(bntTradingLiquidity, tknTradingLiquidity, tknExcess, tknStakedBalance, withdrawalFee, tknWithdrawValue)
    elif b + c > e*(1 - n): 
        if b + c > e and x < hlim(tknTradingLiquidity, tknExcess, tknStakedBalance) and x < hmaxSurplus(tknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee): 
            updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser = arbitrageWithdrawalSurplus(bntTradingLiquidity, tknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee, tknWithdrawValue)
        elif x*(1 - n) <= c: 
            updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser = defaultWithdrawalSurplusCovered(bntTradingLiquidity, tknTradingLiquidity, withdrawalFee, tknWithdrawValue)
        else: 
            updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser = defaultWithdrawalSurplusExposed(bntTradingLiquidity, tknTradingLiquidity, tknExcess, withdrawalFee, tknWithdrawValue)
    elif b + c <= e*(1 - n): 
        if x < hlim(tknTradingLiquidity, tknExcess, tknStakedBalance) and x < hmaxDeficit(tknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee): 
            updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser = arbitrageWithdrawalDeficit(bntTradingLiquidity, tknTradingLiquidity, tknExcess, tknStakedBalance, tradingFee, withdrawalFee, tknWithdrawValue)
        elif x*(1 - n)*(b + c) <= c*e: 
            updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser = defaultWithdrawalDeficitCovered(tknTradingLiquidity, bntTradingLiquidity, tknTradingLiquidity, tknExcess, tknStakedBalance, withdrawalFee, tknWithdrawValue)
        else:
            updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser = defaultWithdrawalDeficitExposed(bntTradingLiquidity, tknTradingLiquidity, tknExcess, tknStakedBalance, withdrawalFee, tknWithdrawValue)
    bntSentToUser, externalProtectionCompensation = externalProtection(bntTradingLiquidity, tknTradingLiquidity, withdrawalFee, bntSentToUser, externalProtectionTknBalance, tknWithdrawValue, tknSentToUser, tradingEnabled)
    print(f"averageTknTradingLiquidity is {b:.18f}, tknExcess is {c:.18f}, tknStakedBalance is {e:.18f}")
    print(f"withdrawalFee is {n:.18f}, and tknWithdrawValue is {x:.18f}")
    print(f"updatedBntLiquidity is {updatedBntLiquidity:.18f}, bntRenounced is {bntRenounced:.18f}, updatedTknLiquidity is {updatedTknLiquidity:.18f}")
    print(f"tknSentToUser is {tknSentToUser:.18f}, bntSentToUser is {bntSentToUser:.18f}, externalProtectionCompensation is {externalProtectionCompensation:.18f}")
    return(updatedBntLiquidity, bntRenounced, updatedTknLiquidity, tknSentToUser, bntSentToUser, externalProtectionCompensation)

def getVirtualBalance(price):
    """
    @Dev
    Takes the price of bnt or tkn as input, and returns the reciprocal.
    This is used during arbitrageTrade, but can also be used during enableTrading (during bootstrapping of a pool). 
    """
    virtualBalance = Decimal("1")/price
    return(virtualBalance)

def processArbitrageTrade(tokenName, tokenPrice, bntPrice, bntTradingLiquidity, tknTradingLiquidity, tradingFee, userTkn, userBnt):
    """
    @Dev
    Takes tokenPrice, bntPrice, bntTradingLiquidity, tknTradingLiquidity, swapFee, tokenName, userTkn, userBnt as inputs. 
    Computes the appropriate arbitrage trade on the tokenName pool. 
    Returns the tradeAmount, sourceToken, and targetToken swap information to close the arbitrage opportunity.  
    Also returns userCapability == True if the user has sufficient funds to close the opportunity, else returns userCapability == False.
    """
    print("Analyzing the apparent arbitrage opportunity.")
    a = bntTradingLiquidity
    b = tknTradingLiquidity
    m = tradingFee
    p = getVirtualBalance(bntPrice)
    q = getVirtualBalance(tokenPrice)
    bntTradeAmount = (-Decimal("2")*a*q + b*m*p + ((Decimal("2")*a*q - b*m*p)**Decimal("2") - Decimal("4")*a*q*(a*q - b*p))**(Decimal("1")/Decimal("2")))/(Decimal("2")*q)
    tknTradeAmount = (-Decimal("2")*b*p + a*m*q + ((Decimal("2")*b*p - a*m*q)**Decimal("2") - Decimal("4")*b*p*(b*p - a*q))**(Decimal("1")/Decimal("2")))/(Decimal("2")*p)
    if bntTradeAmount > 0:
        sourceToken = "bnt"
        targetToken = tokenName
        tradeAmount = bntTradeAmount
        userCapability = userBnt > bntTradeAmount
    elif tknTradeAmount > 0:
        sourceToken = tokenName
        targetToken = "bnt"
        tradeAmount = tknTradeAmount
        userCapability = userTkn > tknTradeAmount
    print(f"the price of {tokenName} is ${tokenPrice:.18f}, the price of bnt is ${bntPrice:.18f}.")
    print(f"The {tokenName}TradingLiquiduty is {tknTradingLiquidity:.18f} and the bntTradingLiquidity is {bntTradingLiquidity:.18f}.")
    print("An arbitrage opportunity was identified.")
    print(f"The opportunity is closed by swapping {tradeAmount:.18f} {sourceToken} for {targetToken}.")
    print(f"userCapability is {userCapability}")
    return(tradeAmount, sourceToken, targetToken, userCapability)

def tradeTknToEma(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, futureEma):
    """
    @Dev
    Takes bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, futureEma as inputs.
    Outputs the tknAmount that should be traded to force the ema and the spot price together on a given pool.
    """
    print("Calculating the number of tkn to trade for bnt.")
    a = bntTradingLiquidity
    b = tknTradingLiquidity
    d = tradingFee
    e = networkFee
    f = futureEma
    tknAmount = ((a*d*(Decimal("1") - e) - Decimal("2")*f*b) + (a*(Decimal("4")*f*b*(Decimal("1") - d*(Decimal("1") - e)) + a*d**Decimal("2")*(Decimal("1") - e)**Decimal("2")))**(Decimal("1")/Decimal("2")))/(Decimal("2")*f)
    print(f"{tknAmount:.18f} tkn need to be traded for bnt.")
    return(tknAmount)

def tradeBntToEma(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, futureEma):
    """
    @Dev
    Takes bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, futureEma as inputs.
    Outputs the bntAmount that should be traded to force the ema and the spot price together on a given pool.
    """
    print("Calculating the number of bnt to trade for tkn.")
    a = bntTradingLiquidity
    b = tknTradingLiquidity
    d = tradingFee
    e = networkFee
    f = futureEma
    x = ((-Decimal("2")*a + b*d*f + ((Decimal("2")*a - b*d*f)**Decimal("2") - Decimal("4")*a*(a - b*f))**(Decimal("1")/Decimal("2")))/Decimal("2"))
    a_recursion = ((a*(a + x) + d*(Decimal("1") - e)*(a*x + x**Decimal("2")))/(a + d*x))
    b_recursion = (b*(a + d*x)/(a + x))
    n = 0
    p = Decimal("0.001")
    while a_recursion/b_recursion < f:
        n += 1
        p += Decimal("0.0001")
        x += (x*(f**p-(a_recursion/b_recursion)**p)/f)
        a_recursion = ((a*(a + x)+d*(Decimal("1") - e)*(a*x + x**Decimal("2")))/(a + d*x))
        b_recursion = (b*(a + d*x)/(a + x))
        if n > 20000:
            break
    bntAmount = x
    print(f"{bntAmount:.18f} bnt need to be traded for tkn.")
    return(bntAmount)

def processForceMovingAverage(tokenName, userTkn, userbnt):
    """
    @Dev
    Takes tokenName as input. 
    Outputs either tradeBNT or tradeTKN functions, and the correct trade amount to force the ema and the spot price together. 
    """
    print("Processing the forced moving average.")
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    tknTradingLiquidity = nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1]
    tradingFee = nonUsers[f"{tokenName}Pool"][f"tradingFee"][-1]
    networkFee = nonUsers["globalProtocolSettings"]["networkFee"][-1]
    emaRate = nonUsers[f"{tokenName}Pool"]["emaRate"][-1]
    spotRate = nonUsers[f"{tokenName}Pool"]["spotRate"][-1]
    futureEma = updateEma(spotRate, emaRate)
    if emaRate < spotRate:
        sourceToken = tokenName
        targetToken = "bnt"
        tradeAmount = tradeTknToEma(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, futureEma)
        userCapability = userTkn > tradeAmount
    elif emaRate > spotRate:
        sourceToken = "bnt"
        targetToken = tokenName
        tradeAmount = tradeBntToEma(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, futureEma)
        userCapability = userbnt > tradeAmount
    else:
        sourceToken = tokenName
        targetToken = tokenName
        tradeAmount = Decimal("0")
        userCapability = False
    print("Processed the ema force.")
    return(tradeAmount, sourceToken, targetToken, userCapability)

def getIdNumber():
    """
    @Dev
    Context: This function is used to assign an idNumber when commencing a cooldown.  
    Returns the 
    """
    print("Generating a cooldown Id number.")
    idNumber = pendingWithdrawals["idCounter"]
    print(f"Cooldown Id number is {idNumber}.")
    return(idNumber)

def isCoolDownComplete(cooldownTimestamp):
    """
    @Dev
    Takes a user's coolDownState as input. 
    Returns True if the number of elapsed timestamps is sufficient to continue with the withdrawal algorithm.
    """
    print("Checking if the cooldown period has completed.")
    coolDownComplete = timestamp - cooldownTimestamp >= nonUsers["globalProtocolSettings"]["cooldownTime"][-1]
    print(f"Cooldown completed = {coolDownComplete}")
    return(coolDownComplete)

def sufficientVbntCheck(bnbntAmount, user):
    """
    @Dev
    Takes the number of bnbnt pool tokens attempting withdrawal, and the username as input. 
    Returns True if the user has at least the same number of vbnt tokens as bnbnt tokens being withdrawn.  
    """
    print("Checking if the user has sufficient vbnt to perform the withdrawal.")
    sufficientVbnt = users[user]["vbntBalance"][-1] >= bnbntAmount
    print(f"Sufficient vbnt = {sufficientVbnt}")
    return(sufficientVbnt)

def unpackCoolDownState(user, idNumber):
    """
    @Dev
    Introduced to make the withdrawals eaiser to handle.
    """
    coolDownState = pendingWithdrawals[user][idNumber]
    cooldownTimestamp = coolDownState["timsestamp"]
    tokenName = coolDownState["tokenName"]
    pooltokenAmount = coolDownState["poolTokenAmount"]
    withdrawValue = coolDownState["withdrawValue"] 
    return(idNumber, cooldownTimestamp, tokenName, pooltokenAmount, withdrawValue, user)

def getUpdatedBntRemainingFunding(updatedBntFundingLimit, bntFundingAmount):
    """
    @Dev
    Context: This function is called during changeBntFundingLimit of any pool. 
    Takes bntFundingLimit, updatedBntFundingLimit, bntRemainingFunding, and bntFundingAmount as inputs.
    Outputs the updatedBntRemainingFunding.
    """
    updatedBntRemainingFunding = updatedBntFundingLimit - bntFundingAmount
    return(updatedBntRemainingFunding)

def getBntRenounced(updatedBntTradingLiquidity, bntTradingLiquidity):
    """
    @Dev
    Context: This function is called during reduceTradingLiquidity of any pool. 
    Takes updatedBntTradingLiquidity, bntTradingLiquidity as inputs.
    Returns bntRenounced. 
    """
    print("Determining how much bntthe protocol should renounce.")
    bntRenounced = bntTradingLiquidity - updatedBntTradingLiquidity
    return(bntRenounced)

def getUpdatedTknTradingLiquidity(tknTradingLiquidity, emaRate, bntRenounced):
    """
    @Dev
    Context: This function is called during reduceTradingLiquidity of any pool. 
    Takes tknTradingLiquidity, emaRate, bnbntRenounced as inputs.
    Returns the updatedTknTradingLiquidity.
    """
    updatedTknTradingLiquidity = max(tknTradingLiquidity - bntRenounced/emaRate, 0)
    return(updatedTknTradingLiquidity)

def getBootstrapRate(bntVirtualBalance, tknTokenVirtualBalance):
    """
    @Dev
    Takes bntVirtualBalance and tknTokenVirtualBalance as inputs. 
    Returns bntVirtualBalance/tknTokenVirtualBalance.
    """
    virtualRate = bntVirtualBalance/tknTokenVirtualBalance
    return(virtualRate)

def getBurnReward(vortexLedgerBalance):
    """
    @Dev
    Context: This function is called during a vortexBurner function call. 
    Takes bntAmount, from the vortexLedger, as an input. 
    Returns the burnReward and the updatedBntAmount.
    """
    print("Calculating the burn reward.")
    burnReward = min(Decimal("100"), vortexLedgerBalance/Decimal("10"))
    bntAmount = vortexLedgerBalance - burnReward
    print(f"The vortexLedgerBalance is {vortexLedgerBalance:.18f}")
    print(f"The burnReward is {burnReward:.18f} and the bntAmount is {bntAmount:.18f}")
    return(burnReward, bntAmount)

def getBntDelta(bntTradingLiquidity, updatedBntLiquidity):
    """
    @Dev
    Calculates the change in the masterVault bnt balance during arbitrage withdrawals.
    """
    bntDelta = updatedBntLiquidity - bntTradingLiquidity
    return(bntDelta)

# Main Functions

In [76]:
getcontext().prec = 155

def doNothing(*args):
    printer()
    return(None)

def newTimestamp(time = 0):
    """
    @Dev
    Increments the timestamp number, a global variable, by 1 seconds.
    """
    global timestamp
    if time == 0:
        seconds = random.choice([1, 2, 3])
        timestamp += seconds 
        print(f"Added a new timestamp; the last blocktime was {seconds} seconds")
    else:
        timestamp = time    
    return(None)

def copyRows():
    print(f"making a new row")
    """
    @Dev
    Creates a new row for all dictionaries and nested dictionaries.
    Subsequent functions then modify the copied row as needed. 
    """
    for faction in factions:
        for entity in faction:
                for key in faction[entity]:
                    if key == "timestamp":
                        faction[entity]["timestamp"].append(timestamp)
                    else:
                        faction[entity][key].append(faction[entity][key][-1])
    return(None)

def protocolBntCheck():
    """
    @Dev
    Asserts that the total bnt balance of the system is equal to the total trading liquidity + vortex ledger bnt balance.
    """
    totalBntTradingLiquidity = sum(nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] for tokenName in whitelistedTokens if tokenName != "bnt")
    vortexledgerBntBalance = (nonUsers["vortexLedger"]["bntBalance"][-1])
    masterVaultBntBalance = (nonUsers["masterVault"]["bntBalance"][-1])
    protocolBntDiscrepancy = (masterVaultBntBalance - vortexledgerBntBalance - totalBntTradingLiquidity)
    assert Decimal(f"{protocolBntDiscrepancy:.8f}") == 0, f"The master vault bnt balance does not agree with the total bnt trading liquidity: protocolBntDiscrepancy = {protocolBntDiscrepancy:.18f}"
    return(None)

def poolShutdownCheck(tokenName):
    """
    @Dev
    Takes tokenName as an input. 
    Checks that the bntMinTradingLiquidity threshold has not been breached. 
    This function is called after changes in the bnt funding limits of a pool, after tkn deposits, and before tkn withdrawals.
    This function returns nothing. 
    """
    print(f"Checking the minimum liquidity requirements for the {tokenName}Pool.")
    bntMinLiquidity = nonUsers["globalProtocolSettings"]["bntMinLiquidity" ][-1]
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    tradingEnabled = nonUsers[f"{tokenName}Pool"]["tradingEnabled"][-1]
    print(f"bntMinLiquidity is {bntMinLiquidity:.18f} and bntTradingLiquidity is {bntTradingLiquidity:.18f}")
    if bntTradingLiquidity < bntMinLiquidity and tradingEnabled == True:
        poolShutdown(tokenName)
    else:
        pass
    return(None)

def poolShutdown(tokenName):
    """
    @Dev
    Takes tokenName as an input. 
    This function is called when the bntMinTradingLiquidity threshold is breached. 
    All bnt trading liquidty is renounced, and the pool is disabled for trading. 
    This function returns nothing. 
    """
    print(f"The {tokenName}Pool was shut down.")
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    stakedBnt = nonUsers["stakingLedger"]["bntBalance"][-1]
    bnbntSupply = nonUsers["erc20Contracts"]["bnbntBalance"][-1]
    bnbntRate = getBnbntRate(stakedBnt, bnbntSupply)
    bnbntRenounced = getBnbntAmount(bnbntRate, bntTradingLiquidity)
    nonUsers[f"{tokenName}Pool"]["tradingEnabled"][-1] = False
    nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] = Decimal("0")
    nonUsers["stakingLedger"]["bntBalance"][-1] -= bntTradingLiquidity
    nonUsers["masterVault"]["bntBalance"][-1] -= bntTradingLiquidity
    nonUsers["erc20Contracts"]["bnbntBalance"][-1] -= bnbntRenounced
    nonUsers["protocolWallet"]["bnbntBalance"][-1] -= bnbntRenounced
    nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1] -= bntTradingLiquidity
    nonUsers[f"{tokenName}Pool"]["bntRemainingFunding"][-1] += bntTradingLiquidity
    nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] = Decimal("0")
    return(None)

def protocolOwnedLiquidity(bntAmount):
    """
    @Dev
    This function handles the adjustments to the system resulting from the protocol minting BNT. 
    The bnbnt supply, and the protocolWallet bnbntBalance are incremented commensurate with the bnbnt/bnt rate, and the input bntAmount. 
    This function returns nothing.
    """
    print("Handling the protocol-owned liquidity.")
    stakedBnt = nonUsers["stakingLedger"]["bntBalance"][-1]
    bnbntSupply = nonUsers["erc20Contracts"]["bnbntBalance"][-1]
    bnbntRate = getBnbntRate(stakedBnt, bnbntSupply)
    bnbntAmount = getBnbntAmount(bnbntRate, bntAmount)
    nonUsers["masterVault"]["bntBalance"][-1] += bntAmount
    nonUsers["erc20Contracts"]["bnbntBalance"][-1] += bnbntAmount
    nonUsers["stakingLedger"]["bntBalance"][-1] += bntAmount
    nonUsers["protocolWallet"]["bnbntBalance"][-1] += bnbntAmount
    return(None)

def depositTKN(tknAmount, tokenName, user):
    """
    @Dev
    Takes an input of tkn and updates the tkn balances of the user, masterVault, and stakingLedger.
    The user is issued bntkn, therefore the bntkn balances are changed for its erc20 contract and the user.
    If allowed, the tkn and bnt trading liquidities of thef {tkn}Pool are also updated.
    If the trading liquidities are updated, the bntRemainingFunding, and bntFunding amounts are updated. 
    If the trading liquidities are updated, the protocol is issued bntkn. 
    If the protocol is issued bntkn, then the bntkn balances are changed for its erc20 contract and the protocolWallet.
    This function returns nothing.
    """
    tknAmount = Decimal(tknAmount)
    print(f"Depositing {tknAmount} {tokenName}")
    stakedTkn = nonUsers["stakingLedger"][f"{tokenName}Balance"][-1]
    bntknSupply = nonUsers["erc20Contracts"][f"bn{tokenName}Balance"][-1]
    bntknRate = getBntknRate(stakedTkn, bntknSupply)
    print('** bntknRate',bntknRate)
    bntknAmount = getBntknAmount(bntknRate, tknAmount)
    print('** bntknAmount',bntknAmount)
    users[user][f"{tokenName}Balance"][-1] -= tknAmount
    users[user][f"bn{tokenName}Balance"][-1] += bntknAmount
    nonUsers["erc20Contracts"][f"bn{tokenName}Balance"][-1] += bntknAmount
    nonUsers["masterVault"][f"{tokenName}Balance"][-1] += tknAmount
    nonUsers["stakingLedger"][f"{tokenName}Balance"][-1] += tknAmount
    tradingEnabled = nonUsers[f"{tokenName}Pool"]["tradingEnabled"][-1]
    spotRate = nonUsers[f"{tokenName}Pool"]["spotRate"][-1]
    emaRate  = nonUsers[f"{tokenName}Pool"]["emaRate"][-1]
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    tknTradingLiquidity = nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] # temporary
    masterVaultTknBalance = nonUsers["masterVault"][f"{tokenName}Balance"][-1]
    bntRemainingFunding = nonUsers[f"{tokenName}Pool"]["bntRemainingFunding"][-1]
    bntIncrease, tknIncrease = poolDepthAdjustment(tradingEnabled, 
                                                    spotRate, 
                                                    emaRate, 
                                                    bntTradingLiquidity,
                                                    tknTradingLiquidity, #temporary
                                                    masterVaultTknBalance, 
                                                    bntRemainingFunding)
    nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] += bntIncrease
    nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] += tknIncrease
    nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1] += bntIncrease
    nonUsers[f"{tokenName}Pool"]["bntRemainingFunding"][-1] -= bntIncrease
    protocolOwnedLiquidity(bntIncrease)
    updatedBntLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    updatedTknLiquidity = nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1]
    newSpotRate = getSpotRate(updatedBntLiquidity, updatedTknLiquidity)
    nonUsers[f"{tokenName}Pool"]["spotRate"][-1] = newSpotRate
    poolShutdownCheck(tokenName)
    protocolBntCheck()
    printer()
    return(None)

def depositBNT(bntAmount, user):
    """
    @Dev
    Takes an input of bnt and updates the bnt balance of the user, then issues the user bnbnt from the protocol reservoir.
    The user's bnt balance is reduced, their bnbnt balance is increased, and the protocolWallet bnbnt balance is decreased. 
    This function returns nothing.
    """
    bntAmount = Decimal(bntAmount)
    print(f"Depositing {bntAmount} bnt")
    stakedBnt = nonUsers["stakingLedger"]["bntBalance"][-1]
    bnbntSupply = nonUsers["erc20Contracts"]["bnbntBalance"][-1]
    bnbntRate = getBnbntRate(stakedBnt, bnbntSupply)
    bnbntAmount = getBnbntAmount(bnbntRate, bntAmount)
    users[user]["bntBalance"][-1] -= bntAmount
    users[user]["bnbntBalance"][-1] += bnbntAmount
    users[user]["vbntBalance"][-1] += bnbntAmount
    nonUsers["protocolWallet"]["bnbntBalance"][-1] -= bnbntAmount
    printer()
    return(None)

def handleEma(tokenName):
    """
    @Dev
    Takes updateAllowed, a bool, and newEma as inputs. 
    Handles the updating of the emaRate, called before a swap is performed.
    This function returns nothing.
    """
    print("Handling the ema.")
    lastSpot = nonUsers[f"{tokenName}Pool"]["spotRate"][-1]
    lastEma = nonUsers[f"{tokenName}Pool"]["emaRate"][-1]
    lastEmaCompressedNumerator = nonUsers[f"{tokenName}Pool"]["emaCompressedNumerator"][-1]
    lastEmaCompressedDenominator = nonUsers[f"{tokenName}Pool"]["emaCompressedDenominator"][-1]
    emaLastUpdated = nonUsers[f"{tokenName}Pool"]["emaLastUpdated"][-1]
    updateAllowed = isEmaUpdateAllowed(emaLastUpdated)
    if updateAllowed == True:
        newEma = updateEma(lastSpot, lastEma)
        NewEmaCompressedNumerator, NewEmaCompressedDenominator = updateCompressedEma(lastSpot, lastEmaCompressedNumerator, lastEmaCompressedDenominator)
        nonUsers[f"{tokenName}Pool"]["emaLastUpdated"][-1] = timestamp
        nonUsers[f"{tokenName}Pool"]["emaRate"][-1] = newEma
        nonUsers[f"{tokenName}Pool"]["emaCompressedNumerator"][-1] = NewEmaCompressedNumerator
        nonUsers[f"{tokenName}Pool"]["emaCompressedDenominator"][-1] = NewEmaCompressedDenominator
        emaDeviation = measureEmaDeviation(newEma, NewEmaCompressedNumerator, NewEmaCompressedDenominator)
        nonUsers[f"{tokenName}Pool"]["emaDeviation"][-1] = emaDeviation
    else:
        pass
    return(None)

def trade(swapAmount, sourceToken, targetToken, user):
    """
    @Dev
    Takes sourceToken, targetToken, swapAmount, user as inputs. 
    The user's bnt and tkn balances, and the trading liquidities, are updated according to the swap algorithm. 
    Two different fee quantities are collected, one denominated in tkn and the other in bnt. 
    The tkn fee is added to the stakingLedger, the bntFee is added to the staking ledger, and the vortexFee fee is added to the vortexLedger. 
    This function returns nothing.
    """
    swapAmount = Decimal(swapAmount)
    print(f"Swapping {swapAmount:.18f} {sourceToken} for {targetToken}.")
    if sourceToken == "bnt" and isTradingEnabled(targetToken) == True:
        targetSentToUser = tradeBntForTkn(swapAmount, targetToken)
    elif targetToken == "bnt" and isTradingEnabled(sourceToken) == True:
        targetSentToUser = tradeTknforBnt(swapAmount, sourceToken)
    elif sourceToken != "bnt" and targetToken != "bnt" and isTradingEnabled(sourceToken) == True and isTradingEnabled(targetToken) == True:
        intermediateBnt = tradeTknforBnt(swapAmount, sourceToken)
        targetSentToUser = tradeBntForTkn(intermediateBnt, targetToken)
    else:
        print("Trading is disabled.")
        swapAmount = Decimal("0")
        targetSentToUser = Decimal("0")
    users[user][f"{sourceToken}Balance"][-1] -= swapAmount
    users[user][f"{targetToken}Balance"][-1] += targetSentToUser
    print(f"{user} swapped {swapAmount:.18f} {sourceToken} for {targetSentToUser:.18f} {targetToken}.")
    protocolBntCheck()
    printer()
    return(None)

def arbitrageTrade(tokenName, user):
    """
    @Dev
    Takes tokenName and user as inputs. 
    This function calls a support function, processArbitrageTrade to analyze the apparent arbitrage opportunity on tokenName.
    If the user has sufficient funds to close the arbitrage opportunity, calls trade with the appropriate amounts. 
    This function returns nothing.
    """
    tokenPrice, bntPrice = getPrices(tokenName, timestamp)
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    tknTradingLiquidity = nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1]
    tradingFee = nonUsers[f"{tokenName}Pool"]["tradingFee"][-1]
    userTkn = users[user][f"{tokenName}Balance"][-1]
    userBnt = users[user]["bntBalance"][-1]
    if isTradingEnabled(tokenName) == True:
        tradeAmount, sourceToken, targetToken, userCapability = processArbitrageTrade(tokenName, tokenPrice, bntPrice, bntTradingLiquidity, tknTradingLiquidity, tradingFee, userTkn, userBnt)
        if userCapability == True:
            trade(tradeAmount, sourceToken, targetToken, user)
        else:
            print("The user has insufficient funds to close the arbitrage opportunity.")
            pass
    else:
        print("Trading is disabled")
        pass  
    return(None)

def forceMovingAverage(tokenName, user):
    """
    @Dev
    Takes tokenName and user as inputs. 
    This function calls a support function, processForceMovingAverage, which itself calls further supporting functions, tradeTknToEma and tradeBntToEma.
    Calling this function will analyze the state of any pool, and create a swap that drives the ema and the spot price together. 
    This function returns nothing.
    """
    print(f"{user} is forcing the ema on the {tokenName}Pool")
    userTkn = users[user][f"{tokenName}Balance"][-1]
    userbnt = users[user]["bntBalance"][-1]
    if isTradingEnabled(tokenName) == True:
        tradeAmount, sourceToken, targetToken, userCapability = processForceMovingAverage(tokenName, userTkn, userbnt)
        if userCapability == True:
            trade(tradeAmount, sourceToken, targetToken, user)
        else:
            print("The user has insufficient funds to force the ema.")
            pass
    else:
        print("Trading is disabled")
        pass  
    return(None)

def beginCooldown(withdrawValue, tokenName, user):
    """
    @Dev
    Takes the username and a quantity of tkn tokens as inputs. 
    The users bntkn is converted into its tkn equivalent, and these values are stored in the pendingWithdrawals with the current timestamp number.
    After a fixed time duration, these items can be retrieved and passed to the withdrawal algorithm.      
    """
    withdrawValue = Decimal(withdrawValue)
    print(f"beginning cooldown on {withdrawValue:.18f} {tokenName}")
    stakedAmount = nonUsers["stakingLedger"][f"{tokenName}Balance"][-1]
    poolTokenSupply = nonUsers["erc20Contracts"][f"bn{tokenName}Balance"][-1]
    poolTokenAmount = (lambda a, b, c: a*b/c)(poolTokenSupply, withdrawValue, stakedAmount)
    idNumber = getIdNumber()
    coolDownState = {
        "timsestamp" : timestamp,
        "tokenName" : tokenName,
        "poolTokenAmount" : poolTokenAmount,
        "withdrawValue" : withdrawValue
        }
    pendingWithdrawals[user][idNumber] = coolDownState
    pendingWithdrawals["idCounter"] = str(int(idNumber) + 1)
    users[user][f"bn{tokenName}Balance"][-1] -= poolTokenAmount
    protocolBntCheck()
    printer()
    return(None)

def withdraw(user, idNumber):
    """
    @Dev
    Takes the username and a quantity of bntkn tokens as inputs. 
    The users bntkn is converted into its tkn equivalent, and this amount is passed to a complex algorithm. 
    The user may receive only tkn, or a mixture of tkn and bnt, depending on the state of the system.
    The staked balances of both bnt and tkn, and the supplies of both bnbnt and bntkn may change. 
    This function returns nothing.
    """
    idNumber, cooldownTimestamp, tokenName, pooltokenAmount, withdrawValue, user = unpackCoolDownState(user, idNumber)
    print(f"Attempting to withdraw {withdrawValue:.18f} {tokenName}")
    withdrawalFee = nonUsers["globalProtocolSettings"]["withdrawalFee"][-1]
    coolDownComplete = isCoolDownComplete(cooldownTimestamp)
    if tokenName != "bnt":
        poolShutdownCheck(tokenName)
        tradingEnabled = nonUsers[f"{tokenName}Pool"]["tradingEnabled"][-1]
        stakedBnt = nonUsers["stakingLedger"]["bntBalance"][-1]
        bnbntSupply = nonUsers["erc20Contracts"]["bnbntBalance"][-1]
        bnbntRate = getBnbntRate(stakedBnt, bnbntSupply)
        emaRate = nonUsers[f"{tokenName}Pool"]["emaRate"][-1]
        spotRate = nonUsers[f"{tokenName}Pool"]["spotRate"][-1]
        if coolDownComplete == True and isPriceStable(spotRate, emaRate) == True:
            print("Withdrawing")
            del pendingWithdrawals[user][idNumber]
            bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
            tknTradingLiquidity = nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1]
            averageTknTradingLiquidity = getAverageTknTradingLiquidity(bntTradingLiquidity, emaRate)
            masterVaultTknBalance = nonUsers["masterVault"][f"{tokenName}Balance"][-1] 
            tknExcess = getTknExcess(masterVaultTknBalance, tknTradingLiquidity) # Justify this later - should it be (b + c) - b or (b_average + c) - b
            tknStakedBalance = nonUsers["stakingLedger"][f"{tokenName}Balance"][-1]
            tradingFee = nonUsers[f"{tokenName}Pool"]["tradingFee"][-1]
            externalProtectionTknBalance = nonUsers[f"{tokenName}Pool"]["externalProtectionVaultBalance"][-1]
            (
                updatedBntLiquidity, 
                bntRenounced, 
                updatedTknLiquidity, 
                tknSentToUser, 
                bntSentToUser, 
                externalProtectionCompensation) = processWithdrawal(
                    tknTradingLiquidity, 
                    bntTradingLiquidity, 
                    averageTknTradingLiquidity, 
                    tknExcess, 
                    tknStakedBalance, 
                    tradingFee, 
                    withdrawalFee, 
                    externalProtectionTknBalance, 
                    withdrawValue, 
                    tradingEnabled)
            if bntRenounced == 0:
                bntDelta = getBntDelta(bntTradingLiquidity, updatedBntLiquidity)
                nonUsers["masterVault"]["bntBalance"][-1] += bntDelta
            nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] = updatedBntLiquidity
            nonUsers["stakingLedger"]["bntBalance"][-1] -= bntRenounced
            nonUsers["masterVault"]["bntBalance"][-1] -= bntRenounced
            nonUsers[f"{tokenName}Pool"]["bntRemainingFunding"][-1] += bntRenounced
            nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1] -= bntRenounced
            bnbntRenounced = getBnbntAmount(bnbntRate, bntRenounced)
            nonUsers["erc20Contracts"]["bnbntBalance"][-1] -= bnbntRenounced
            nonUsers["protocolWallet"]["bnbntBalance"][-1] -= bnbntRenounced
            nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] = updatedTknLiquidity
            # users[user][f"bn{tokenName}Balance"][-1] -= pooltokenAmount
            nonUsers["erc20Contracts"][f"bn{tokenName}Balance"][-1] -= pooltokenAmount
            nonUsers["stakingLedger"][f"{tokenName}Balance"][-1] -= withdrawValue
            nonUsers["masterVault"][f"{tokenName}Balance"][-1] -= tknSentToUser
            users[user][f"{tokenName}Balance"][-1] += tknSentToUser + externalProtectionCompensation
            nonUsers[f"{tokenName}Pool"]["externalProtectionVaultBalance"][-1] -= externalProtectionCompensation
            users[user]["bntBalance"][-1] += bntSentToUser
            newSpotRate = getSpotRate(updatedBntLiquidity, updatedTknLiquidity)
            nonUsers[f"{tokenName}Pool"]["spotRate"][-1] = newSpotRate
        else:
            print("Couldn't withdraw")
            pass
    else:
        if coolDownComplete == True and sufficientVbntCheck(pooltokenAmount, user) == True:
            del pendingWithdrawals[user][idNumber]
            bntAmount = getBntAmount(withdrawValue, withdrawalFee)
            # users[user]["bnbntBalance"][-1] -= pooltokenAmount
            users[user]["vbntBalance"][-1] -= pooltokenAmount
            users[user]["bntBalance"][-1] += bntAmount
            nonUsers["protocolWallet"]["bnbntBalance"][-1] += pooltokenAmount
        else:
            print("Couldn't withdraw")
            pass
    protocolBntCheck()
    printer()
    return(None)

def enableTrading(tokenName, bntVirtualBalance, tknTokenVirtualBalance, bntFundingLimit):
    """
    @Dev
    Takes a virtual balance of bnt and tkn as input, as if from an external price feed; user input is an empty string.
    Checks if the masterVault has the minimum bnt equivalent of tkn to justify bootstrapping. 
    If the requirements are met, the trading liquidities are increased to twice the minimum threshold. 
    Since bnt is minted to the pool, the bntFundingAmount and bntRemainingFunding are adjusted accordingly.
    The protocolWallet bnbntBalance is also adjusted commensurate with the current rate of bnbnt/bnt.
    After bootstrapping, the spotRate == emaRate == virtualRate. 
    This function returns nothing.
    """
    print("Enabling trading")
    bntVirtualBalance = Decimal(bntVirtualBalance)
    tknTokenVirtualBalance = Decimal(tknTokenVirtualBalance)
    bntFundingLimit = Decimal(bntFundingLimit)
    bootstrapRate = getBootstrapRate(bntVirtualBalance, tknTokenVirtualBalance)
    bntMinLiquidity = nonUsers["globalProtocolSettings"]["bntMinLiquidity"][-1]
    vaultTknBalance = nonUsers["masterVault"][f"{tokenName}Balance"][-1]
    bntBootstrapLiquidity = twiceMinimumLiquidity(bntMinLiquidity)
    tknBootstrapLiquidity = getTknBootstrapLiquidity(bntBootstrapLiquidity, bootstrapRate)
    requirementsMet = testMinimumLiquidity(vaultTknBalance, tknBootstrapLiquidity)
    if requirementsMet == True:
        nonUsers[f"{tokenName}Pool"]["tradingEnabled"][-1] = True
        nonUsers[f"{tokenName}Pool"]["bntFundingLimit"][-1] = bntFundingLimit
        nonUsers[f"{tokenName}Pool"]["spotRate"][-1] = nonUsers[f"{tokenName}Pool"]["emaRate"][-1] = bootstrapRate
        nonUsers[f"{tokenName}Pool"]["emaCompressedNumerator"][-1] = bntVirtualBalance
        nonUsers[f"{tokenName}Pool"]["emaCompressedDenominator"][-1] = tknTokenVirtualBalance
        nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] = bntBootstrapLiquidity
        nonUsers[f"{tokenName}Pool"]['bntFundingAmount'][-1] = bntBootstrapLiquidity
        nonUsers[f"{tokenName}Pool"]['bntRemainingFunding'][-1] = bntFundingLimit - bntBootstrapLiquidity
        nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] = tknBootstrapLiquidity
        protocolOwnedLiquidity(bntBootstrapLiquidity)
    else:
        pass
    protocolBntCheck()
    printer()
    return(None)

def changeTradingFee(tokenName, newTradingFee):
    """
    @Dev
    Takes tokenName and newFee as inputs. 
    Adjusts the trading fee of the associated pool.
    This function returns nothing.
    """
    newTradingFee = Decimal(newTradingFee)
    nonUsers[f"{tokenName}Pool"]["tradingFee"][-1] = newTradingFee
    print(f"Changed the trading fee on {tokenName} to {newTradingFee:.18f}")
    printer()
    return(None)

def changeNetworkFee(newNetworkFee):
    """
    @Dev
    Takes newNetworkFee as input. 
    Adjusts the network fee in the global settings.
    This function returns nothing.
    """
    newNetworkFee = Decimal(newNetworkFee)
    nonUsers["globalProtocolSettings"]["networkFee"][-1] = newNetworkFee
    print(f"Changed the network fee (i.e vortex fee) to {newNetworkFee:.18f}")
    return(None)

def changeWithdrawalFee(newWithdrawalFee):
    """
    @Dev
    Takes newWithdrawalFee as input. 
    Adjusts the newWithdrawalFee in the global settings.
    This function returns nothing.
    """
    newWithdrawalFee = Decimal(newWithdrawalFee)
    nonUsers["globalProtocolSettings"]["withdrawalFee"][-1] = newWithdrawalFee
    print(f"Changed the withdrawal fee to {newWithdrawalFee:.18f}")
    return(None)

def changeBntFundingLimit(tokenName, updatedBntFundingLimit):
    """
    @Dev
    Takes tokenName, updatedBntFundingLimit as inputs.
    Updates the bntFundingLimit of the appropriate pool.
    This function returns nothing.
    """
    updatedBntFundingLimit = Decimal(updatedBntFundingLimit)
    bntFundingAmount = nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1]
    print(f"Changing bntFundingAmount from {bntFundingAmount:.18f} to {updatedBntFundingLimit:.18f}")
    if updatedBntFundingLimit > bntFundingAmount:
        updatedBntRemainingFunding = getUpdatedBntRemainingFunding(updatedBntFundingLimit, bntFundingAmount)
        nonUsers[f"{tokenName}Pool"]["bntFundingLimit"][-1] = updatedBntFundingLimit
        nonUsers[f"{tokenName}Pool"]["bntRemainingFunding"][-1] = updatedBntRemainingFunding
    else:
        print("The updatedBntFundingLimit cannot be less than the bntFundingAmount.")
        print("Transaction reverted.")
        doNothing()
    print(f"Successfully changed the bntFundingAmount to {updatedBntFundingLimit:.18f}")
    return(None)



def reduceTradingLiquidity(tokenName, updatedBntTradingLiquidity):
    """
    @Dev
    Takes tokenName, updatedBntTradingLiquidity as inputs. 
    Updates the state of the appropriate pool, and the protocol holdings, as required.
    This function returns nothing.
    """
    updatedBntTradingLiquidity = Decimal(updatedBntTradingLiquidity)
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    tknTradingLiquidity = nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1]
    emaRate = nonUsers[f"{tokenName}Pool"]["emaRate"][-1]
    spotRate = nonUsers[f"{tokenName}Pool"]["spotRate"][-1]
    bntRenounced = getBntRenounced(updatedBntTradingLiquidity, bntTradingLiquidity)
    stakedBnt = nonUsers["stakingLedger"]["bntBalance"][-1]
    bnbntSupply = nonUsers["erc20Contracts"]["bnbntBalance"][-1]
    bnbntRate = getBnbntRate(stakedBnt, bnbntSupply)
    bnbntRenounced = getBnbntAmount(bnbntRate, bntRenounced)
    updatedTknTradingLiquidity = getUpdatedTknTradingLiquidity(tknTradingLiquidity, emaRate, bntRenounced)
    if isPriceStable(spotRate, emaRate) == False:
        doNothing()
    else:
        nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1] -= bntRenounced
        nonUsers["stakingLedger"]["bntBalance"][-1] -= bntRenounced
        nonUsers["masterVault"]["bntBalance"][-1] -= bntRenounced
        nonUsers["erc20Contracts"]["bnbntBalance"][-1] -= bnbntRenounced
        nonUsers["protocolWallet"]["bnbntBalance"][-1] -= bnbntRenounced
        nonUsers[f"{tokenName}Pool"]["bntFundingLimit"][-1] = updatedBntTradingLiquidity
        nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1] = updatedBntTradingLiquidity
        nonUsers[f"{tokenName}Pool"]["bntRemainingFunding"][-1] = Decimal("0")
        nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1] = updatedTknTradingLiquidity
        poolShutdownCheck(tokenName)
        print(f"Reducing the trading liquidity on the {tokenName} pool from {bntTradingLiquidity:.18f} to {updatedBntTradingLiquidity:.18f} bnt.")
        print(f"The protocol is renouncing {bnbntRenounced:.18f} bnt.")
        print(f"The protocol is renouncing {bnbntRenounced:.18f} bnt, and moving {updatedTknTradingLiquidity - tknTradingLiquidity:.18f} tkn off-curve.")
    protocolBntCheck()
    printer()
    return(None)

def vortexBurner(user):
    """
    @Dev
    Takes the username as input.
    The vortex burner actions are carried out; the bntBalance of the vortexLedger is swapped for vbnt, and the vbnt is destroyed.  
    The user that triggers the vortex is rewarded; their bntBalance is increased slighty. 
    This function returns nothing.
    """
    print(f"{user} has triggered the vortex burner.")
    vortexLedgerBalance = nonUsers["vortexLedger"]["bntBalance"][-1]   
    if isTradingEnabled("vbnt") == True and vortexLedgerBalance > 0:
        burnReward, bntAmount = getBurnReward(vortexLedgerBalance)     
        print(f"Trading {bntAmount:.18f} bnt for vbnt")
        handleEma("vbnt")
        bntTradingLiquidity = nonUsers["vbntPool"]["bntTradingLiquidity"][-1]
        tknTradingLiquidity = nonUsers["vbntPool"]["vbntTradingLiquidity"][-1]
        tradingFee = nonUsers["vbntPool"]["tradingFee"][-1]
        networkFee = Decimal("0")
        direction = "bnt"
        updatedBntLiquidity = getChangedBntTradingLiquidity(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, bntAmount, direction)
        updatedTknLiquidity = getChangedTknTradingLiquidity(bntTradingLiquidity, tknTradingLiquidity, tradingFee, bntAmount, direction)
        vbntToBurner = getTargetAmount(bntTradingLiquidity, tknTradingLiquidity, tradingFee, bntAmount, direction)
        tradeFee = swapFeeCollection(bntTradingLiquidity, tknTradingLiquidity, tradingFee, networkFee, bntAmount, direction)
        users[user]["bntBalance"][-1] += burnReward
        nonUsers["vortexLedger"]["bntBalance"][-1] = Decimal("0")
        nonUsers["vortexLedger"]["vbntBurned"][-1] += vbntToBurner
        nonUsers["masterVault"]["bntBalance"][-1] -= burnReward
        nonUsers["masterVault"]["vbntBalance"][-1] -= vbntToBurner
        nonUsers["vbntPool"]["bntTradingLiquidity"][-1] = updatedBntLiquidity
        nonUsers["vbntPool"]["vbntTradingLiquidity"][-1] = updatedTknLiquidity
        nonUsers["stakingLedger"]["vbntBalance"][-1] += tradeFee
        newSpotRate = getSpotRate(updatedBntLiquidity, updatedTknLiquidity)
        nonUsers["vbntPool"]["spotRate"][-1] = newSpotRate
        print(f"The vortex destroyed {vbntToBurner} vbnt.")
        print(f"{user} has received {burnReward:.18f} bnt as a reward.")
    else:
        print("The vortex burner cannot be activated at this time.")
        pass
    printer()
    return(None)




# Printer

In [77]:
getcontext().prec = 155

def printer():
    print("\n")
    print("***TRADING LIQUIDITIES***")
    print()
    for tokenName in whitelistedTokens:
        if tokenName != "bnt":
            print(f"{tokenName.upper()} POOL")
            print(f"bntTradingLiquidity : {nonUsers[f'{tokenName}Pool']['bntTradingLiquidity'][-1]:.18f}")
            print(f"{tokenName}TradingLiquidity : {nonUsers[f'{tokenName}Pool'][f'{tokenName}TradingLiquidity'][-1]:.18f}")
            print()
    print("\n")
    print("***USER WALLETS***")
    print()
    for userName in users:
        print(f"{userName.upper()}")
        for tokenName in [f(tokenName) for tokenName in whitelistedTokens for f in (lambda tokenName: f"{tokenName}Balance", lambda tokenName: f"bn{tokenName}Balance")]:
            print(f"{tokenName} : {users[userName][f'{tokenName}'][-1]:.18f}")
            print()
    print(f"BANCOR PROTOCOL")
    print(f"bnbnt : {nonUsers['protocolWallet']['bnbntBalance'][-1]:.18f}")
    print()
    print("\n")
    print("***VAULT BALANCES***")
    print()
    for tokenName in whitelistedTokens:
        print(f"{tokenName} : {nonUsers['masterVault'][f'{tokenName}Balance'][-1]:.18f}")
        print()
    print("\n")
    print("***VORTEX LEDGER***")
    print(f"bnt : {nonUsers['vortexLedger'][f'bntBalance'][-1]:.18f}")

    print("\n")
    print("***STAKING LEDGERS***")
    print()
    for tokenName in whitelistedTokens:
        print(f"{tokenName} : {nonUsers['stakingLedger'][f'{tokenName}Balance'][-1]:.18f}")
        print()
    print("\n")
    print("***POOL TOKEN SUPPLIES***")
    print()
    for tokenName in whitelistedTokens:
        print(f"bn{tokenName} : {nonUsers['erc20Contracts'][f'bn{tokenName}Balance'][-1]:.18f}")
        print()
    print('end of action sequence')
    print('\n')
    print('***************')
    print('\n')

# Simulation Support Functions

In [78]:
getcontext().prec = 155

def randomiseAmounts(amount):
    """
    @Dev
    For simulation purposes only. 
    """
    if random.randint(0, 1000) != 0:
        maxAmount = amount*Decimal("0.0001")
        minAmount = amount*Decimal("0.000001")
    else:
        maxAmount = amount*Decimal("0.01")
        minAmount = amount*Decimal("0.001")
    randomAmount = Decimal(random.uniform(float(minAmount), float(maxAmount)))
    return(randomAmount)

def getMasterVaultTVL(masterVaultBalance, tokenPrice):
    """
    @Dev
    Simulation purposes only.
    """
    masterVaultTVL = masterVaultBalance*tokenPrice
    return(masterVaultTVL)

def getMaximumTknDeposit(masterVaultTVL, targetTVL, userFunds):
    """
    @Dev
    Simulation purposes only.
    """
    maximumTknDeposit = max(targetTVL - masterVaultTVL, userFunds)
    return(maximumTknDeposit)

def isProtocolBnbntHealthy(protocolBnbnt, bnbntSupply):
    """
    @Dev
    Simulation purposes only.
    """
    protocolBnbntHealthy = protocolBnbnt/bnbntSupply > Decimal("0.5")
    return(protocolBnbntHealthy)
    
def getMaximumBntDeposit(bnbntSupply, protocolBnbnt, bnbntRate, userBnt):
    """
    @Dev
    Simulation purposes only.
    """
    maximumBnbntIssued = bnbntSupply/Decimal("0.5") - protocolBnbnt
    maximumBntDeposit = max(maximumBnbntIssued/bnbntRate, userBnt)
    return(maximumBntDeposit)

def getMinimumBntFunding(bntMinLiquidity):
    """
    @Dev
    Simulation purposes only.
    """
    smallestRansomisation = Decimal("0.000001")
    bntBootstrapLiquidity = twiceMinimumLiquidity(bntMinLiquidity)
    smallestBntFundingLimit = bntBootstrapLiquidity/smallestRansomisation
    return(smallestBntFundingLimit) 

def randomCooldownAmount(userBntknAmount):
    """
    @Dev
    Simulation purposes only.
    """
    if random.randint(0, 10) != 0:
        maxAmount = userBntknAmount*Decimal("0.1")
        minAmount = userBntknAmount*Decimal("0.01")
    else:
        maxAmount = userBntknAmount*Decimal("1")
        minAmount = userBntknAmount*Decimal("0.5")
    bntknAmount = Decimal(random.uniform(float(minAmount), float(maxAmount)))
    return(bntknAmount)

def randomNewTradingFee(tradingFee):
    """
    @Dev
    Simulation purposes only.
    """
    minTradingFee = max(tradingFee/Decimal("3"), Decimal("0.001"))
    maxTradingFee = min(tradingFee*Decimal("3"), Decimal("0.05"))
    newTradingFee = Decimal(random.uniform(float(minTradingFee), float(maxTradingFee)))
    return(newTradingFee)

def randomNewNetworkFee(networkFee):
    """
    @Dev
    Simulation purposes only.
    """
    minNetworkFee = max(networkFee/Decimal("2"), Decimal("0.05"))
    maxNetworkFee = min(networkFee*Decimal("2"), Decimal("0.25"))
    newNetworkFee = Decimal(random.uniform(float(minNetworkFee), float(maxNetworkFee)))
    return(newNetworkFee)

def randomNewWithdrawalFee(withdrawalFee):
    """
    @Dev
    Simulation purposes only.
    """
    minWithdrawalFee = max(withdrawalFee/Decimal("2"), Decimal("0.001"))
    maxWithdrawalFee = min(withdrawalFee*Decimal("2"), Decimal("0.01"))
    newNetworkFee = Decimal(random.uniform(float(minWithdrawalFee), float(maxWithdrawalFee)))
    return(newNetworkFee)

def randomNewBntFundingLimit(bntFundingAmount):
    """
    @Dev
    Simulation purposes only.
    """
    minBntFundingLimit = bntFundingAmount*Decimal("1.5")
    maxBntFundingLimit = bntFundingAmount*Decimal("3.0")
    newBntFundingLimit = Decimal(random.uniform(float(minBntFundingLimit), float(maxBntFundingLimit)))
    return(newBntFundingLimit)

def randomNewBntTradingLiquidity(bntTradingLiquidity):
    """
    @Dev
    Simulation purposes only.
    """
    minNewBntTradingLiquidity = bntTradingLiquidity*Decimal("0.4")
    maxNewBntTradingLiquidity = bntTradingLiquidity*Decimal("0.9")
    newNewBntTradingLiquidity = Decimal(random.uniform(float(minNewBntTradingLiquidity), float(maxNewBntTradingLiquidity)))
    return(newNewBntTradingLiquidity)


# Main Simulation Functions

In [79]:
getcontext().prec = 155

def randomTrades():
    """
    @Dev
    Simulation purposes only. 
    """
    for i in range(random.randint(1, 10)):
        user = random.choice([user for user in users])
        tokens = [token for token in whitelistedTokens]
        sourceToken = tokens.pop(tokens.index(random.choice(tokens)))
        targetToken = random.choice(tokens)
        sourceLiquidity = nonUsers[f"{sourceToken if sourceToken != 'bnt' else targetToken}Pool"][f"{sourceToken}TradingLiquidity"][-1]
        userFunds = users[user][f"{sourceToken}Balance"][-1]
        for i in range(random.randint(1, 5)):
            swapAmount = randomiseAmounts(sourceLiquidity)
            if userFunds > swapAmount:
                trade(swapAmount, sourceToken, targetToken, user)
            else: 
                trade(userFunds, sourceToken, targetToken, user)
            copyRows()
            if random.randint(0, 2) != 0:
                newTimestamp()
    return(None)



def randomDeposits():
    """
    @Dev
    Simulation purposes only.
    """
    for i in range(random.randint(1, 10)):
        user = random.choice([user for user in users])
        tokenName = random.choice(whitelistedTokens)
        userTkn = users[user][f"{tokenName}Balance"][-1]
        userBnt = users[user]["bntBalance"][-1]
        bnbntSupply = nonUsers["erc20Contracts"]["bnbntBalance"][-1]
        protocolBnbnt = nonUsers["protocolWallet"]["bnbntBalance"][-1]
        stakedBnt = nonUsers["stakingLedger"]["bntBalance"][-1]
        bnbntRate = getBnbntRate(stakedBnt, bnbntSupply)
        userFunds = users[user][f"{tokenName}Balance"][-1]
        if tokenName != "bnt":
            masterVaultBalance = nonUsers["masterVault"][f"{tokenName}Balance"][-1]
            tokenPrice = getPrices(tokenName, timestamp)[0]
            masterVaultTVL = getMasterVaultTVL(masterVaultBalance, tokenPrice)
            targetTVL = Decimal("100000000")
            if masterVaultTVL < targetTVL:
                maximumTknDeposit = getMaximumTknDeposit(masterVaultTVL, targetTVL, userTkn)
                depositAmount = randomiseAmounts(maximumTknDeposit)
                if depositAmount < userFunds:
                    depositTKN(depositAmount, tokenName, user)
                else:
                    pass
            else:
                pass
        elif tokenName == "bnt":
            if bnbntSupply > 0 and isProtocolBnbntHealthy(protocolBnbnt, bnbntSupply):
                maximumBntDeposit = getMaximumBntDeposit(bnbntSupply, protocolBnbnt, bnbntRate, userBnt)
                depositAmount = randomiseAmounts(maximumBntDeposit)
                if depositAmount < userFunds:
                    depositBNT(depositAmount, user)
                else:
                    pass
            else:
                pass
        else:
            pass
        copyRows()
        if random.randint(0, 2) != 0:
            newTimestamp()
    return(None)

def randomArbitrageTrades():
    """
    @Dev
    Simulation purposes only.
    """
    for i in range(random.randint(1, 10)):
        user = random.choice([user for user in users])
        tokens = [token for token in whitelistedTokens if token != "bnt"]
        tokenName = random.choice(tokens)
        if isTradingEnabled(tokenName) == True:
            arbitrageTrade(tokenName, user)
            copyRows()
            if random.randint(0, 2) != 0:
                newTimestamp()
        else:
            pass
    return(None)

def randomEmaForceTrades():
    """
    @Dev
    Simulation purposes only.
    """
    for i in range(random.randint(1, 10)):
        user = random.choice([user for user in users])
        tokens = [token for token in whitelistedTokens if token != "bnt"]
        tokenName = random.choice(tokens)
        if isTradingEnabled(tokenName) == True:
            forceMovingAverage(tokenName, user)
            copyRows()
            if random.randint(0, 2) != 0:
                newTimestamp()
        else:
            pass
    return(None)
    
def randomEnableTrading():
    """
    @Dev
    Simulation purposes only.
    """
    tokens = [token for token in whitelistedTokens if token != "bnt"]
    for tokenName in tokens:
        tradingEnabled = nonUsers[f"{tokenName}Pool"]["tradingEnabled"][-1]
        bntMinLiquidity = nonUsers["globalProtocolSettings"]["bntMinLiquidity"][-1]
        if tradingEnabled == False:
            tokenPrice, bntPrice = getPrices(tokenName, timestamp)
            bntVirtualBalance = getVirtualBalance(bntPrice)
            tknTokenVirtualBalance = getVirtualBalance(tokenPrice)
            smallestBntFundingLimit = getMinimumBntFunding(bntMinLiquidity)
            bntFundingLimit = randomiseAmounts(smallestBntFundingLimit)
            enableTrading(tokenName, bntVirtualBalance, tknTokenVirtualBalance, bntFundingLimit)
            copyRows()
            if random.randint(0, 2) != 0:
                newTimestamp()
        else:
            pass
    return(None)


def randomBeginCooldowns():
    """
    @Dev
    Simulation purposes only.
    """
    for i in range(random.randint(1, 10)):
        user = random.choice([user for user in users])
        tokenName = random.choice(whitelistedTokens)
        userBntknAmount = users[user][f"bn{tokenName}Balance"][-1]
        if userBntknAmount > 0:
            bntknAmount = randomCooldownAmount(userBntknAmount)
            stakedTkn = nonUsers["stakingLedger"][f"{tokenName}Balance"][-1]
            bntknSupply = nonUsers["erc20Contracts"][f"bn{tokenName}Balance"][-1]
            bntknRate = getBntknRate(stakedTkn, bntknSupply)
            withdrawValue = getTknAmount(bntknRate, bntknAmount)
            beginCooldown(withdrawValue, tokenName, user)
            copyRows()
            if random.randint(0, 2) != 0:
                newTimestamp()
        else:
            pass
    return(None)

def randomWithdrawals():
    """
    @Dev
    Simulation purposes only.
    """
    for i in range(random.randint(1, 10)):
        user = random.choice([user for user in users])
        if len(pendingWithdrawals[user]) > 0:
            idNumber = random.choice([idNumber for idNumber in pendingWithdrawals[user]])
            withdraw(user, idNumber)
            copyRows()
            if random.randint(0, 2) != 0:
                newTimestamp()
        else:
            pass
    return(None)

def randomChangeTradingFee():
    """
    @Dev
    Simulation purposes only.
    """
    for i in range(random.randint(1, 3)):
        tokenName = random.choice(whitelistedTokens)
        tradingFee = nonUsers[f"{tokenName}Pool"]["tradingFee"][-1]
        newTradingFee = randomNewTradingFee(tradingFee)
        changeTradingFee(tokenName, newTradingFee)
        copyRows()
        if random.randint(0, 2) != 0:
            newTimestamp()
    return(None)

def randomChangeNetworkFee():
    """
    @Dev
    Simulation purposes only.
    """
    networkFee = nonUsers["globalProtocolSettings"]["networkFee"][-1]
    newNetworkFee = randomNewNetworkFee(networkFee)
    changeNetworkFee(newNetworkFee)
    copyRows()
    if random.randint(0, 2) != 0:
        newTimestamp()
    return(None)

def randomChangeWithdrawalFee():
    """
    @Dev
    Simulation purposes only.
    """
    withdrawalFee = nonUsers["globalProtocolSettings"]["withdrawalFee"][-1]
    newWithdrawalFee = randomNewWithdrawalFee(withdrawalFee)
    changeNetworkFee(newWithdrawalFee)
    copyRows()
    if random.randint(0, 2) != 0:
        newTimestamp()
    return(None)

def randomChangeBntFundingLimit():
    """
    @Dev
    Simulation purposes only.
    """
    tokenName = random.choice(whitelistedTokens)
    bntFundingAmount = nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1]
    updatedBntFundingLimit = randomNewBntFundingLimit(bntFundingAmount)
    changeBntFundingLimit(tokenName, updatedBntFundingLimit)
    copyRows()
    if random.randint(0, 2) != 0:
        newTimestamp()
    return(None)

def randomReduceTradingLiquidity():
    """
    @Dev
    Simulation purposes only.
    """
    tokenName = random.choice(whitelistedTokens)
    bntTradingLiquidity = nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1]
    if bntTradingLiquidity > 0:
        updatedBntTradingLiquidity = randomNewBntTradingLiquidity(bntTradingLiquidity)
        reduceTradingLiquidity(tokenName, updatedBntTradingLiquidity)
        copyRows()
        if random.randint(0, 2) != 0:
            newTimestamp()
    else:
        pass
    return(None)

def randomVortexBurner():
    """
    @Dev
    Simulation purposes only.
    """
    user = random.choice([user for user in users])
    vortexBurner(user)
    copyRows()
    if random.randint(0, 2) != 0:
        newTimestamp()
    return(None)

def actions():
    """
    @Dev
    Simulation purposes only.
    These probabilities are completely arbitrary.
    """
    i = random.randint(0,2000) 
    if i < 400:
        randomTrades()
    elif 400 < i < 1000:
        randomArbitrageTrades()
    elif 1000 < i < 1200:
        randomEmaForceTrades()
    elif 1200 < i < 1400:
        randomDeposits()
    elif 1400 < i < 1500:
        randomBeginCooldowns()
    elif 1500 < i < 1700:
        randomWithdrawals()
    elif 1700 < i < 1710:
        randomChangeTradingFee()
    elif 1710 < i < 1720:
        randomChangeNetworkFee()
    elif 1720 < i < 1730:
        randomChangeWithdrawalFee() 
    elif 1730 < i < 1740:
        randomReduceTradingLiquidity()
    elif 1740 < i < 1800:
        randomChangeBntFundingLimit()
    else:
        randomVortexBurner()
    randomEnableTrading()

def monteCarloSimulation():
    """
    @Dev
    Simulation purposes only.
    Performs a monte carlo simulation of Bancor 3.
    """
    global timestamp
    while timestamp <= priceFeeds.shape[0]:
        actions()

# Security Monitor Support Functions

In [80]:
def contractConvertPPM(numberPPM):
    """
    @Dev
    Takes a number in ppm.
    Returns the same number in decimal.
    """
    decimalNumber = Decimal(numberPPM)/Decimal("1000000")
    return(decimalNumber)

def getContractBlockNumber(df, index):
    """
    @Dev
    Gets the blocknumber recorded in df[index] for all events. 
    """
    blockNumber = df["blocknumber"][index]
    return(blockNumber)

def getContractFeePPM(df, index):
    """
    @Dev
    Gets the newFeePPM recorded in df[index] during contractNetworkFeePPMUpdated and contractTradingFeePPMUpdated.
    """
    newFeePPM = Decimal(df["newFeePPM"][index])
    return(newFeePPM)

def getTxHash(df, index):
    """
    @Dev
    Gets the tokenAmount in df[index] for all events. 
    """
    txHash = df["txhash"][index]
    return(txHash)

def getContractTokenName(df, index):
    """
    @Dev
    Gets the tokenName from the tokenAddress recorded in df[index] during contractTradingFeePPMUpdated and contractTradingEnabled.
    """
    try:
        tokenAddress = str(df["pool"][index])
        tokenName = contractTokenDictionary[tokenAddress]
    except KeyError:
        tokenName = str(df["tokenSymbol"][index])
    return(tokenName)

def getContractNewStatus(df, index):
    """
    @Dev
    Gets the newStatus (True/False) recorded in df[index] during contractTradingEnabled.
    """
    newStatus = bool(df["newStatus"][index])
    return(newStatus)

def getContractLastBntFundingLimit(tokenName):
    """
    @Dev
    The enableTrading function used in this simulator takes bntFundingLimit a one of its arguments. 
    However, apparently on solidity the FundingLimits function is called on a prior block, then TradingEnabled ona later one. 
    Therefore, the FundingLimits is not associated with the transaction hash of TradingEnabled, and pulling the required arguments from both is problematic. 
    Rather than change the existing simulation, instead a work-around is used. 
    During FundingLimits in the contracts, this simulator calls changeBntFundingLimit on the appropriate block. 
    This function is used to recall the bntFundingLimit later, and then passes it to enableTrading on the next block. 
    """
    bntFundingLimit = nonUsers[f"{tokenName}Pool"]["bntFundingLimit"][-1]
    return(bntFundingLimit)

def getContractBntNewLiquidity(df, index):
    """
    @Dev
    Gets the bntNewLiquidity in df[index] during contractTradingEnabled and trading.
    """
    bntVirtualBalance = Decimal(df["bntNewLiquidity"][index])/Decimal("10")**Decimal("18")
    return(bntVirtualBalance)

def getContractTknNewLiquidity(df, index, tokenDecimals):
    """
    @Dev
    Gets the tknNewLiquidity in df[index] during contractTradingEnabled and trading.
    """
    tknNewLiquidity = Decimal(df["tknNewLiquidity"][index])/Decimal(10**tokenDecimals)
    return(tknNewLiquidity)

def getContractFundingLimit(df, index):
    """
    @Dev
    Gets the newLimit in df[index] during contractFundingLimits.
    """
    updatedBntFundingLimit = Decimal(df["newLimit"][index])/Decimal(10**18)
    return(updatedBntFundingLimit)

def getContractTokenDecimals(df, index):
    """
    @Dev
    Gets the tknDecimal in df[index] during contractTradingEnabled.
    """
    tokenDecimals = Decimal(df["tknDecimals"][index])
    return(tokenDecimals)

def getContractTokenAmount(df, index, tokenDecimals):
    """
    @Dev
    Gets the tokenAmount in df[index] during contractDepositTkn.
    """
    tknAmount = Decimal(df["tokenAmount"][index])/Decimal(10**tokenDecimals)
    return(tknAmount)

def getContractReserveTokenAmount(df, index, tokenDecimals):
    """
    @Dev
    Gets the reserveTokenAmount in df[index] during contractWithdrawalInitiated.
    """
    withdrawValue = Decimal(df["reserveTokenAmount"][index])/Decimal(10**tokenDecimals)
    return(withdrawValue)

def getContractBntAmount(df, index):
    """
    @Dev
    Gets the bntAmount in df[index] during contractDepositBnt and duplicateTradeCheck.
    """
    bntAmount = Decimal(df["bntAmount"][index])/Decimal(10**18)
    return(bntAmount)

def getContractSourceToken(df, index):
    """
    @Dev
    Gets the sourceToken in df[index] during contractTrade.
    """
    tokenAddress = df["sourceToken"][index]
    tokenName = contractTokenDictionary[tokenAddress]
    return(tokenName)

def getContractTargetToken(df, index):
    """
    @Dev
    Gets the targetToken in df[index] during contractTrade.
    """
    tokenAddress = df["targetToken"][index]
    tokenName = contractTokenDictionary[tokenAddress]
    return(tokenName)

def getContractSourceTokenDecimals(df, index):
    """
    @Dev
    Gets the sourceDecimals in df[index] during contractTrade.
    """
    sourceDecimals = Decimal(df["sourceDecimals"][index])
    return(sourceDecimals)

def getContractTargetTokenDecimals(df, index):
    """
    @Dev
    Gets the targetDecimals in df[index] during contractTrade.
    """
    targetDecimals = Decimal(df["targetDecimals"][index])
    return(targetDecimals)

def getContractSourceAmount(df, index, sourceDecimals):
    """
    @Dev
    Gets the sourceAmount in df[index] during contractTrade.
    """
    sourceAmount = Decimal(df["sourceAmount"][index])/Decimal(10**sourceDecimals)
    return(sourceAmount)

def getContractTargetAmount(df, index, targetDecimals):
    """
    @Dev
    Gets the targetAmount in df[index] during contractTrade.
    """
    targetAmount = Decimal(df["targetAmount"][index])/Decimal(10**targetDecimals)
    return(targetAmount)

def getContractContextId(df, index):
    """
    @Dev
    Gets the contextId in df[index] during contractTradeProcessor.
    """
    contextId = df["contextId"][index]
    return(contextId)

def getContractBntAddress(df, index):
    """
    @Dev
    Gets the bntAddress in df[index] during contractTradeProcessor.
    """
    bntAddress = df["bntAddress"][index]
    return(bntAddress)

def getContractTraderAddress(df, index):
    """
    @Dev
    Gets the trader in df[index] during contractTradeProcessor.
    """
    traderAddress = df["trader"][index]
    return(traderAddress)

def getContractTraderAddress(df, index):
    """
    @Dev
    Gets the trader in df[index] during contractTradeProcessor.
    """
    traderAddress = df["trader"][index]
    return(traderAddress)

def getTargetFeeAmount(df, index, targetDecimals):
    """
    @Dev
    Gets the targetFeeAmount in df[index] during contractTradeProcessor.
    """
    targetFeeAmount = Decimal(df["targetFeeAmount"][index])/Decimal("10")**Decimal(targetDecimals)
    return(targetFeeAmount)

def getBntFeeAmountAmount(df, index):
    """
    @Dev
    Gets the bntFeeAmount in df[index] during contractTradeProcessor.
    """
    bntFeeAmount = Decimal(df["bntFeeAmount"][index])/Decimal("10")**Decimal("18")
    return(bntFeeAmount)

def getContractBntPrevLiquidity(df, index):
    """
    @Dev
    Gets the bntPrevLiquidity from the events.
    """
    bntPrevLiquidity = Decimal(df["bntPrevLiquidity"][index])/Decimal("10")**Decimal("18")
    return(bntPrevLiquidity)

def getContractTknPrevLiquidity(df, index, tknDecimals):
    """
    @Dev
    Gets the tknPrevLiquidity from the events.
    """
    tknPrevLiquidity = Decimal(df["tknPrevLiquidity"][index])/Decimal("10")**Decimal(tknDecimals)
    return(tknPrevLiquidity)

# def isFinalIndex(tradesThisHashIndices, index):
#     """
#     @Dev
#     Returns True if the index is the higher of the two indices listed in tradesThisHashIndices.
#     """  
#     finalIndex = index == tradesThisHashIndices[-1]
#     return(finalIndex)
    
# def contractTradeSanityChecks(df, index, txHash):
#     """
#     @Dev
#     Tests some basic assumptions about the underlying logic on which the automation here depends.
#     """
#     tradesThisHashIndices = [txHashIndex for txHashIndex in df.index[df['txhash'] == txHash].tolist() if df["type"][txHashIndex] == "trade"]
#     if not isSingleHop(tradesThisHashIndices) and not isDoubleHop(tradesThisHashIndices):
#         raise ValueError("The trades on this hash cannot be categorized as single- or double-hop")
#     if isDoubleHop(tradesThisHashIndices):
#         index0 = tradesThisHashIndices[0]
#         index1 = tradesThisHashIndices[1] 
#         bntAmount0 = getContractBntAmount(df, index0)
#         bntAmount1 = getContractBntAmount(df, index1)
#         contextId0 = getContractContextId(df, index0)
#         contextId1 = getContractContextId(df, index1)
#         assert index1 - index0 == 1, "Trades have non-consecutive indices"
#         if contextId0 == contextId1:
#             assert bntAmount0 == bntAmount1, f"The bnt amounts are inconsistent: {bntAmount0} and {bntAmount1}"
#     return(tradesThisHashIndices)

def getVarsTknToBnt(df, index):
    """
    @Dev
    Gets sourceToken, sourceDecimals, sourceTokenAmount, targetBntAmount and bntFeeAmount during contractTrades.
    """
    sourceToken = getContractSourceToken(df, index)
    sourceDecimals = getContractSourceTokenDecimals(df, index)
    sourceTokenAmount = getContractSourceAmount(df, index, sourceDecimals)
    targetBntAmount = getContractBntAmount(df, index)
    bntFeeAmount = getBntFeeAmountAmount(df, index)
    return(sourceToken, sourceDecimals, sourceTokenAmount, targetBntAmount, bntFeeAmount)

def getVarsBntToTkn(df, index):
    """
    @Dev
    Gets sourceBntAmount, targetToken, targetDecimals, targetTokenAmount and tknFeeAmount during contractTrades.
    """
    sourceBntAmount = getContractBntAmount(df, index)
    targetToken = getContractTargetToken(df, index)
    targetDecimals = getContractTargetTokenDecimals(df, index)
    targetTokenAmount = getContractTargetAmount(df, index, targetDecimals)
    tknFeeAmount = getTargetFeeAmount(df, index, targetDecimals)
    return(sourceBntAmount, targetToken, targetDecimals, targetTokenAmount, tknFeeAmount)

def getContractLiquidities(df, index, tknDecimals):
    """
    @Dev
    Gets bntPrevLiquidity, bntNewLiquidity, tknPrevLiquidity and tknNewLiquidity during contractTrades.
    """
    bntPrevLiquidity =  getContractBntPrevLiquidity(df, index)
    bntNewLiquidity = getContractBntNewLiquidity(df, index)
    tknPrevLiquidity = getContractTknPrevLiquidity(df, index, tknDecimals)
    tknNewLiquidity = getContractTknNewLiquidity(df, index, tknDecimals)
    return(bntPrevLiquidity, bntNewLiquidity, tknPrevLiquidity, tknNewLiquidity)

def getImpliedTknFees(bntPrevLiquidity, bntNewLiquidity, tknPrevLiquidity, tknNewLiquidity, SourceBntAmount, tknFeeAmount):
    a0 = Decimal(bntPrevLiquidity)
    a1 = Decimal(bntNewLiquidity)
    b0 = Decimal(tknPrevLiquidity)
    b1 = Decimal(tknNewLiquidity)
    f = Decimal(SourceBntAmount)
    h = Decimal(tknFeeAmount)
    impliedTradingFee = h*(a0 + f)/(b0*f)
    impliedNetworkFee = ((h*(a0 + f) + a0*b0 - a1*b1)/(h*(a0 + f)))
    impliedTknFeeToStakingLedger = (a1*b1 - a0*b0)/(a0 + f)
    impliedBntToVortexLedger = (a0 + f)*(a0*b0 + h*(a0 + f) - a1*b1)/(a0*b0 + h*(a0 + f))
    return(impliedTradingFee, impliedNetworkFee, impliedTknFeeToStakingLedger, impliedBntToVortexLedger)

def getImpliedBntFees(bntPrevLiquidity, bntNewLiquidity, targetBntAmount, bntFeeAmount):
    a0 = Decimal(bntPrevLiquidity)
    a1 = Decimal(bntNewLiquidity)
    f = Decimal(targetBntAmount)
    j = Decimal(bntFeeAmount)
    impliedTradingFee = j/(j + f)
    impliedNetworkFee = (a0-a1-f)/j
    impliedBntFeeToStakingLedger = a1 - a0 + j + f 
    impliedBntToVortexLedger = a0 - a1 - f
    return(impliedTradingFee, impliedNetworkFee, impliedBntFeeToStakingLedger, impliedBntToVortexLedger)

def getContractDepositTknOutputs(df, index, tokenDecimals):
    """
    @Dev
    Gets the following outputs from the event, which can be compared with the simulation:
    +------------------------+-------------------------------------------------------------------+
    | Contract Names         | Simulator Names                                                   |
    |------------------------+-------------------------------------------------------------------|
    | PC_TLU_liquidity       | nonUsers['masterVault']['tknBalance'][-1]                         |
    | PC_TLU_stakedBalance   | nonUsers['stakingLedger']['tknBalance'][-1]                       |
    | PC_TLU_poolTokenSupply | nonUsers['erc20Contracts']['bntknBalance'][-1]                    |
    | poolTokenAmount        | users[user]['bntknBalance'][-1] - users[user]['bntknBalance'][-2] |
    +------------------------+-------------------------------------------------------------------+
    Returns: contractMasterVaultTknBalance, contractStakingLedgerTknBalance, contractErc20ContractTknBalance, contractBntknAmount.
    """
    contractMasterVaultTknBalance = Decimal(df["PC_TLU_liquidity"][index])/Decimal(10**tokenDecimals)
    contractStakingLedgerTknBalance = Decimal(df["PC_TLU_stakedBalance"][index])/Decimal(10**tokenDecimals)
    contractErc20ContractTknBalance = Decimal(df["PC_TLU_poolTokenSupply"][index])/Decimal(10**18)
    contractBntknAmount = Decimal(df["poolTokenAmount"][index])/Decimal(10**18)
    return(contractMasterVaultTknBalance, contractStakingLedgerTknBalance, contractErc20ContractTknBalance, contractBntknAmount)

def getSimulationDepositTknOutputs(tokenName):
    """
    @Dev
    Gets the following outputs from the simulator, which can be compared with the events:
    +------------------------+-------------------------------------------------------------------+
    | Contract Names         | Simulator Names                                                   |
    |------------------------+-------------------------------------------------------------------|
    | PC_TLU_liquidity       | nonUsers['masterVault']['tknBalance'][-1]                         |
    | PC_TLU_stakedBalance   | nonUsers['stakingLedger']['tknBalance'][-1]                       |
    | PC_TLU_poolTokenSupply | nonUsers['erc20Contracts']['bntknBalance'][-1]                    |
    | poolTokenAmount        | users[user]['bntknBalance'][-1] - users[user]['bntknBalance'][-2] |
    +------------------------+-------------------------------------------------------------------+
    Returns: simulatorMasterVaultTknBalance, simulatorStakingLedgerTknBalance, simulatorErc20ContractTknBalance, simulatorBntknAmount.
    """
    simulatorMasterVaultTknBalance = nonUsers["masterVault"][f"{tokenName}Balance"][-1]
    simulatorStakingLedgerTknBalance = nonUsers["stakingLedger"][f"{tokenName}Balance"][-1] 
    simulatorErc20ContractTknBalance = nonUsers["erc20Contracts"][f"bn{tokenName}Balance"][-1]
    simulatorBntknAmount = users[user][f"{tokenName}Balance"][-1] - users[user][f"{tokenName}Balance"][-1]
    return(simulatorMasterVaultTknBalance, simulatorStakingLedgerTknBalance, simulatorErc20ContractTknBalance, simulatorBntknAmount)


def getContractWithdrawalInitiatedOutputs(df, index):
    """
    @Dev
    Gets the following outputs from the event, which can be compared with the simulation:
    +------------------+------------------------------------------------------+
    | Contract Names   | Simulator Names                                      |
    |------------------+------------------------------------------------------|
    | poolTokenAmount  | pendingWithdrawls[idNumber][user]['poolTokenAmount'] |
    +------------------+------------------------------------------------------+
    Returns: contractPoolTokenAmount.
    """
    contractPoolTokenAmount = Decimal(df["poolTokenAmount"][index])/Decimal(10**18)
    return(contractPoolTokenAmount)


def getSimulatedWithdrawalInitiatedOutputs():
    """
    @Dev
    Gets the following outputs from the simulator, which can be compared with the events:
    +------------------+------------------------------------------------------+
    | Contract Names   | Simulator Names                                      |
    |------------------+------------------------------------------------------|
    | poolTokenAmount  | pendingWithdrawls[idNumber][user]['poolTokenAmount'] |
    +------------------+------------------------------------------------------+
    Returns: contractPoolTokenAmount.
    """
    idNumber = str(int(pendingWithdrawals["idCounter"]) - 1)
    simulatorPoolTokenAmount = pendingWithdrawals[user][idNumber]['poolTokenAmount']
    return(simulatorPoolTokenAmount)

def getContractDepositBntOutputs(df, index):
    """
    @Dev
    Gets the following outputs from the events, which can be compared with the simulator:
    +------------------+-------------------------------------------------------------------+
    | Contract Names   | Simulator Names                                                   |
    |------------------+-------------------------------------------------------------------|
    | poolTokenAmount  | users[user]['bnbntBalance'][-1] - users[user]['bnbntBalance'][-2] |
    | vbntAmount       | users[user]['vbntBalance'][-1] - users[user]['vbntBalance'][-2]   |
    +------------------+-------------------------------------------------------------------+
    Returns: contractPoolTokenAmount, contractVbntAmount.
    """
    contractPoolTokenAmount = Decimal(df["poolTokenAmount"][index])/Decimal(10**18)
    contractVbntAmount = Decimal(df["vbntAmount"][index])/Decimal(10**18)
    return(contractPoolTokenAmount, contractVbntAmount)

def getSimulatorDepositBntOutputs():
    """
    @Dev
    Gets the following outputs from the simulator, which can be compared with the events:
    +------------------+-------------------------------------------------------------------+
    | Contract Names   | Simulator Names                                                   |
    |------------------+-------------------------------------------------------------------|
    | poolTokenAmount  | users[user]['bnbntBalance'][-1] - users[user]['bnbntBalance'][-2] |
    | vbntAmount       | users[user]['vbntBalance'][-1] - users[user]['vbntBalance'][-2]   |
    +------------------+-------------------------------------------------------------------+
    Returns: simulatorPoolTokenAmount, simulatorVbntAmount.
    """
    simulatorPoolTokenAmount = users[user]["bnbntBalance"][-1] - users[user]["bnbntBalance"][-2]
    simulatorVbntAmount = users[user]["bnbntBalance"][-1] - users[user]["bnbntBalance"][-2]
    return(simulatorPoolTokenAmount, simulatorVbntAmount)

def isBntSource(df, index):
    """
    @Dev
    Returns True if the bntNewLiquidity > bntPrevLiquidity
    """
    bntPrevLiquidity =  getContractBntPrevLiquidity(df, index)
    bntNewLiquidity = getContractBntNewLiquidity(df, index)
    bntSource = bntNewLiquidity > bntPrevLiquidity
    return(bntSource)

def getContractTradeOutputs(df, index, txHash):
    """
    @Dev
    Gets the following outputs from the simulator, which can be compared with the events:
    +----------------------------+-----------------------------------------------------------------------------------------------------------------+
    | Contract Names             | Simulator Names                                                                                                 |
    |----------------------------+-----------------------------------------------------------------------------------------------------------------|
    | targetAmount               | nonUsers['masterVault'][f'{targetToken}Balance'][-2] - nonUsers['masterVault'][f'{targetToken}Balance'][-1]     |
    | bntPrevLiquidity           | nonUsers[f'{tkn}Pool']['bntTradingLiquidity'][-2]                                                               |
    | bntNewLiquidity            | nonUsers[f'{tkn}Pool']['bntTradingLiquidity'][-1]                                                               |
    | tknPrevLiquidity           | nonUsers[f'{tkn}Pool'][f'{tkn}TradingLiquidity'][-2]                                                            |
    | tknNewLiquidity            | nonUsers[f'{tkn}Pool'][f'{tkn}TradingLiquidity'][-1]                                                            |
    | targetFeeAmount            | b*d*f/(a + f)                                                                                                   |
    | bntFeeAmount               | a*d*g/(b + g)                                                                                                   |
    | *impliedTradingFee         | nonUsers[f'{tkn}Pool]['tradingFee'][-1]                                                                         |
    | *impliedNetworkFee         | nonUsers['globalProtocolSettings']['networkFee'][-1]                                                            |
    | *impliedFeeToStakingLedger | nonUsers['stakingLedger'][f'{targetToken}Balance'][-1] - nonUsers['stakingLedger'][f'{targetToken}Balance'][-2] |
    | *impliedBntToVortexLedger  | nonUsers['vortexLedger']['bntBalance'][-1] - nonUsers['vortexLedger']['bntBalance'][-2]                         |
    |                            |                                                                                                                 |
    | MEANING OF *               | * = calculated from events outputs; cannot be read directly                                                     |
    +----------------------------+-----------------------------------------------------------------------------------------------------------------+

    """
    if not isBntSource(df, index):
        targetToken = "bnt"
        sourceToken, sourceDecimals, sourceTokenAmount, targetTokenAmount, feeAmount = getVarsTknToBnt(df, index)
        bntPrevLiquidity, bntNewLiquidity, tknPrevLiquidity, tknNewLiquidity = getContractLiquidities(df, index, sourceDecimals)
        impliedTradingFee, impliedNetworkFee, impliedFeeToStakingLedger, impliedBntToVortexLedger = getImpliedBntFees(bntPrevLiquidity, bntNewLiquidity, targetTokenAmount, feeAmount)  
    else:
        sourceToken = "bnt"
        sourceTokenAmount, targetToken, targetDecimals, targetTokenAmount, feeAmount = getVarsBntToTkn(df, index)
        bntPrevLiquidity, bntNewLiquidity, tknPrevLiquidity, tknNewLiquidity = getContractLiquidities(df, index, targetDecimals)
        impliedTradingFee, impliedNetworkFee, impliedFeeToStakingLedger, impliedBntToVortexLedger = getImpliedTknFees(bntPrevLiquidity, bntNewLiquidity, tknPrevLiquidity, tknNewLiquidity, sourceTokenAmount, feeAmount)
    return(sourceToken, sourceTokenAmount, targetToken, targetTokenAmount, bntPrevLiquidity, bntNewLiquidity, tknPrevLiquidity, tknNewLiquidity, feeAmount, impliedTradingFee, impliedNetworkFee, impliedFeeToStakingLedger, impliedBntToVortexLedger)

def getSimulatedTradingPoolVars(sourceToken, targetToken):
    """
    @Dev
    Gets simulatedBntPrevLiquidity, simulatedBntNewLiquidity, simulatedTknPrevLiquidity and simulatedTknNewLiquidity during contractTrade.
    """
    tkn = targetToken if targetToken != 'bnt' else sourceToken
    simulatedBntPrevLiquidity =  nonUsers[f"{tkn}Pool"]["bntTradingLiquidity"][-2]
    simulatedBntNewLiquidity = nonUsers[f"{tkn}Pool"]["bntTradingLiquidity"][-1]
    simulatedTknPrevLiquidity = nonUsers[f"{tkn}Pool"][f"{tkn}TradingLiquidity"][-2]
    simulatedTknNewLiquidity = nonUsers[f"{tkn}Pool"][f"{tkn}TradingLiquidity"][-1]
    trueTradingFee = nonUsers[f"{tkn}Pool"]["tradingFee"][-1]
    return(trueTradingFee, simulatedBntPrevLiquidity, simulatedBntNewLiquidity, simulatedTknPrevLiquidity, simulatedTknNewLiquidity)

def getSimulatedFeeAmount(sourceToken, sourceTokenAmount, prevBntLiquidity, prevTknLiquidity, trueTradingFee):
    """
    @Dev
    Gets the fee paid by the trader - something that is not explicitly recorded in the standard simulation. 
    """
    if sourceToken == "bnt":
        simulatedFeeAmount = prevTknLiquidity*trueTradingFee*sourceTokenAmount/(prevBntLiquidity + sourceTokenAmount)
    else:
        simulatedFeeAmount = prevBntLiquidity*trueTradingFee*sourceTokenAmount/(prevTknLiquidity + sourceTokenAmount)
    return(simulatedFeeAmount)

def getSimulatedTradingOutputs(targetToken):
    """
    @Dev
    Gets simulatedTargetTokenAmount, simulatedFeeToStakingLedger and simulatedBntToVortexLedger during contractTrade.
    """
    simulatedTargetTokenAmount = nonUsers["masterVault"][f"{targetToken}Balance"][-2] - nonUsers["masterVault"][f"{targetToken}Balance"][-1]
    simulatedFeeToStakingLedger = nonUsers["stakingLedger"][f"{targetToken}Balance"][-1] - nonUsers["stakingLedger"][f"{targetToken}Balance"][-2]
    simulatedBntToVortexLedger = nonUsers["vortexLedger"]["bntBalance"][-1] - nonUsers["vortexLedger"]["bntBalance"][-2]
    return(simulatedTargetTokenAmount, simulatedFeeToStakingLedger, simulatedBntToVortexLedger)

# Security Monitor Recorders

In [81]:
def depositTknComparisson(
        blockNumber, 
        txHash, 
        tokenName,
        tknAmount, 
        contractMasterVaultTknBalance, simulatorMasterVaultTknBalance,
        contractStakingLedgerTknBalance, simulatorStakingLedgerTknBalance,
        contractErc20ContractTknBalance, simulatorErc20ContractTknBalance,
        contractBntknAmount, simulatorBntknAmount):
    """
    @Dev
    Adds the results of contractDepositTkn to the monitor["token deposits"] dictionary.
    +---------------------------------+----------------------------------+
    | Contract Names                  | Simulator Names                  |
    |---------------------------------+----------------------------------|
    | contractMasterVaultTknBalance   | simulatorMasterVaultTknBalance   |
    | contractStakingLedgerTknBalance | simulatorStakingLedgerTknBalance |
    | contractErc20ContractTknBalance | simulatorErc20ContractTknBalance |
    | contractBntknAmount             | simulatorBntknAmount             |
    +---------------------------------+----------------------------------+
    """
    monitor["token deposits"]["block number"].append(blockNumber)
    monitor["token deposits"]["transaction hash"].append(txHash)
    monitor["token deposits"]["token"].append(tokenName)
    monitor["token deposits"]["token amount"].append(tknAmount)
    monitor["token deposits"]["ethereum master vault balance"].append(contractMasterVaultTknBalance)
    monitor["token deposits"]["simulated master vault balance"].append(simulatorMasterVaultTknBalance)
    monitor["token deposits"]["ethereum staking ledger balance"].append(contractStakingLedgerTknBalance)
    monitor["token deposits"]["simulated staking ledger balance"].append(simulatorStakingLedgerTknBalance)
    monitor["token deposits"]["ethereum pool token supply"].append(contractErc20ContractTknBalance)
    monitor["token deposits"]["simulated pool token supply"].append(simulatorErc20ContractTknBalance)
    monitor["token deposits"]["ethereum minted pool token amount"].append(contractBntknAmount)
    monitor["token deposits"]["simulated minted pool token amount"].append(simulatorBntknAmount)
    return(None)


def withdrawalInitiatedComparisson(
    blockNumber, 
    txHash, 
    tokenName, 
    withdrawValue, 
    contractPoolTokenAmount, simulatorPoolTokenAmount):
    """
    @Dev
    Adds the results of contractWithdrawalInitiated to the monitor["withdrawals initiated"] dictionary.
    +-------------------------+--------------------------+
    | Contract Names          | Simulator Names          |
    |-------------------------+--------------------------|
    | contractPoolTokenAmount | simulatorPoolTokenAmount |
    +-------------------------+--------------------------+
    """
    monitor["withdrawals initiated"]["block number"].append(blockNumber)
    monitor["withdrawals initiated"]["transaction hash"].append(txHash)
    monitor["withdrawals initiated"]["token"].append(tokenName)
    monitor["withdrawals initiated"]["withdrawn token amount"].append(withdrawValue)
    monitor["withdrawals initiated"]["ethereum pool token amount"].append(contractPoolTokenAmount)
    monitor["withdrawals initiated"]["simulated pool token amount" ].append(simulatorPoolTokenAmount)
    return(None)


def depositBntComparisson(
    blockNumber, 
    txHash, 
    bntAmount,
    contractPoolTokenAmount, simulatorPoolTokenAmount,
    contractVbntAmount, simulatorVbntAmount):
    """
    @Dev
    Adds the results of contractDepositBnt to the monitor["bnt deposits"] dictionary.
    +-------------------------+--------------------------+
    | Contract Names          | Simulator Names          |
    |-------------------------+--------------------------|
    | contractPoolTokenAmount | simulatorPoolTokenAmount |
    | contractVbntAmount      | simulatorVbntAmount      |
    +-------------------------+--------------------------+
    """
    monitor["bnt deposits"]["block number"].append(blockNumber)
    monitor["bnt deposits"]["transaction hash"].append(txHash)
    monitor["bnt deposits"]["bnt amount"].append(txHash)
    monitor["bnt deposits"]["ethereum transfered bnbnt amount"].append(contractPoolTokenAmount)
    monitor["bnt deposits"]["simulated transfered bnbnt amount"].append(simulatorPoolTokenAmount)
    monitor["bnt deposits"]["ethereum minted vbnt amount"].append(contractVbntAmount)
    monitor["bnt deposits"]["simulated minted vbnt amount"].append(simulatorVbntAmount)
    return(None)

def tradeComparisson(
    blockNumber, 
    txHash, 
    sourceTokenAmount, 
    sourceToken, 
    targetToken, 
    contractTargetTokenAmount, simulatedTargetTokenAmount, 
    contractBntPrevLiquidity, simulatedBntPrevLiquidity, 
    contractTknPrevLiquidity, simulatedTknPrevLiquidity, 
    contractBntNewLiquidity, simulatedBntNewLiquidity, 
    contractTknNewLiquidity, simulatedTknNewLiquidity, 
    impliedTradingFee, trueTradingFee, 
    impliedNetworkFee, trueNetworkFee, 
    contractFeeAmount, simulatedFeeAmount, 
    impliedFeeToStakingLedger, simulatedFeeToStakingLedger, 
    impliedBntToVortexLedger, simulatedBntToVortexLedger):
    """
    @Dev
    Adds the results of contractTrade to the monitor["trade activity"] dictionary.
    +---------------------------+-----------------------------+
    | Contract Names            | Simulator Names             |
    |---------------------------+-----------------------------|
    | contractTargetTokenAmount | simulatedTargetTokenAmount  |
    | contractBntPrevLiquidity  | simulatedBntPrevLiquidity   |
    | contractTknPrevLiquidity  | simulatedTknPrevLiquidity   |
    | contractBntNewLiquidity   | simulatedBntNewLiquidity    |
    | contractTknNewLiquidity   | simulatedTknNewLiquidity    |
    | impliedTradingFee         | trueTradingFee              |
    | impliedNetworkFee         | trueNetworkFee              |
    | contractFeeAmount         | simulatedFeeAmount          |
    | impliedFeeToStakingLedger | simulatedFeeToStakingLedger |
    | impliedBntToVortexLedger  | simulatedBntToVortexLedger  |
    +---------------------------+-----------------------------+
    """
    monitor["trades"]["block number"].append(blockNumber)
    monitor["trades"]["transaction hash"].append(txHash)
    monitor["trades"]["source token amount"].append(sourceTokenAmount)
    monitor["trades"]["source token"].append(sourceToken)
    monitor["trades"]["target token"].append(targetToken)
    monitor["trades"]["ethereum target token amount"].append(contractTargetTokenAmount)
    monitor["trades"]["simulated target token amount"].append(simulatedTargetTokenAmount)
    monitor["trades"]["ethereum previous bnt liquidity"].append(contractBntPrevLiquidity)
    monitor["trades"]["simulated previous bnt liquidity"].append(simulatedBntPrevLiquidity)
    monitor["trades"]["ethereum previous tkn liquidity"].append(contractTknPrevLiquidity)
    monitor["trades"]["simulated previous tkn liquidity"].append(simulatedTknPrevLiquidity)
    monitor["trades"]["ethereum new bnt liquidity"].append(contractBntNewLiquidity)
    monitor["trades"]["simulated new bnt liquidity"].append(simulatedBntNewLiquidity)
    monitor["trades"]["ethereum new tkn liquidity"].append(contractTknNewLiquidity)
    monitor["trades"]["simulated new tkn liquidity"].append(simulatedTknNewLiquidity)
    monitor["trades"]["ethereum implied trading fee"].append(impliedTradingFee)
    monitor["trades"]["true trading fee"].append(trueTradingFee)
    monitor["trades"]["ethereum implied network fee"].append(impliedNetworkFee)
    monitor["trades"]["true network fee"].append(trueNetworkFee)
    monitor["trades"]["ethereum fee paid by trader"].append(contractFeeAmount)
    monitor["trades"]["simulated fee paid by trader"].append(simulatedFeeAmount)
    monitor["trades"]["ethereum implied fee to staking ledger"].append(impliedFeeToStakingLedger)
    monitor["trades"]["simulated fee to staking ledger"].append(simulatedFeeToStakingLedger)
    monitor["trades"]["ethereum implied bnt to vortex ledger"].append(impliedBntToVortexLedger)
    monitor["trades"]["simulated bnt to vortex ledger"].append(simulatedBntToVortexLedger)
    return(None)

In [82]:
def getContracttokenDictionary():
    """
    @Dev
    Returns the tokenContract : TKN dictionary.
    """
    contractTokenDictionary = {
        "0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE" : "eth",
        "0x6B175474E89094C44Da98b954EedeAC495271d0F" : "dai",
        "0x514910771AF9Ca656af840dff83E8264EcF986CA" : "link", 
        "0x1F573D6Fb3F13d689FF844B4cE37794d79a7FF1C" : "bnt"
    }
    return(contractTokenDictionary)

def getEmptyMonitor():
    """
    @Dev
    Returns an empty monitor dictionary.
    """
    monitor = {
        "token deposits" : {
            "block number" : [],
            "transaction hash" : [],
            "token" : [],
            "token amount" : [],
            "ethereum master vault balance" :[],
            "simulated master vault balance" : [],
            "ethereum staking ledger balance" :[],
            "simulated staking ledger balance" : [],
            "ethereum pool token supply" :[],
            "simulated pool token supply" : [],
            "ethereum minted pool token amount" : [],
            "simulated minted pool token amount" : [],
        },
        "withdrawals initiated" :{
            "block number" : [],
            "transaction hash" : [],
            "token" : [],
            "withdrawn token amount": [],
            "ethereum pool token amount" : [],
            "simulated pool token amount" : []
        },
        "bnt deposits" : {
            "block number" : [],
            "transaction hash" : [],
            "bnt amount" : [],
            "ethereum transfered bnbnt amount" : [],
            "simulated transfered bnbnt amount" : [],
            "ethereum minted vbnt amount" : [],
            "simulated minted vbnt amount" : [],
        },
        "trades" : {
            "block number" : [],
            "transaction hash" : [],
            "source token amount" : [],
            "source token" : [],
            "target token" : [],
            "ethereum target token amount" : [],
            "simulated target token amount" : [],
            "ethereum previous bnt liquidity" : [],
            "simulated previous bnt liquidity" : [],
            "ethereum previous tkn liquidity" : [],
            "simulated previous tkn liquidity" : [],
            "ethereum new bnt liquidity" : [],
            "simulated new bnt liquidity" : [],
            "ethereum new tkn liquidity" : [],
            "simulated new tkn liquidity" : [],
            "ethereum implied trading fee" : [],
            "true trading fee" : [],
            "ethereum implied network fee" : [],
            "true network fee" : [],
            "ethereum fee paid by trader" : [],
            "simulated fee paid by trader" : [],
            "ethereum implied fee to staking ledger" : [],
            "simulated fee to staking ledger" : [],
            "ethereum implied bnt to vortex ledger" : [],
            "simulated bnt to vortex ledger" : [],
        }
    }
    return(monitor)


# Main Security Monitoring Functions

In [83]:

def contractNetworkFeePPMUpdated(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the newFeePPM event from the contracts.
    """
    newFeePPM = getContractFeePPM(df, index)
    newNetworkFee = contractConvertPPM(newFeePPM)
    changeNetworkFee(newNetworkFee)
    return(None)

def contractTradingFeePPMUpdated(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the TradingFeePPMUpdated event from the contracts.
    """
    newFeePPM = getContractFeePPM(df, index)
    newTradingFee = contractConvertPPM(newFeePPM)
    tokenName = getContractTokenName(df, index)
    changeTradingFee(tokenName, newTradingFee)
    return(None)

def contractDepositingEnabled(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the DepositingEnabled event from the contracts.
    """
    doNothing()
    return(None)

def contractFundingLimits(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the FundingLimits event from the contracts.
    """
    tokenName = getContractTokenName(df, index)
    updatedBntFundingLimit = getContractFundingLimit(df, index)
    changeBntFundingLimit(tokenName, updatedBntFundingLimit)
    return(None)

def contractTradingEnabled(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the DepositingEnabled event from the contracts.
    """
    newStatus = getContractNewStatus(df, index)
    if newStatus == True:
        tokenName = getContractTokenName(df, index)
        tokenDecimals = getContractTokenDecimals(df, index)
        bntVirtualBalance = getContractBntNewLiquidity(df, index)
        tknTokenVirtualBalance = getContractTknNewLiquidity(df, index, tokenDecimals)
        bntFundingLimit = getContractLastBntFundingLimit(tokenName)
        enableTrading(tokenName, bntVirtualBalance, tknTokenVirtualBalance, bntFundingLimit)
    else:
        doNothing()
    return(None)

def contractDepositTkn(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the depositTKN event from the contracts.
    """
    tokenName = getContractTokenName(df, index)
    tokenDecimals = getContractTokenDecimals(df, index)
    tknAmount = getContractTokenAmount(df, index, tokenDecimals)
    (
        contractMasterVaultTknBalance, 
        contractStakingLedgerTknBalance, 
        contractErc20ContractTknBalance, 
        contractBntknAmount) = getContractDepositTknOutputs(df, index, tokenDecimals)
    newTimestamp(blockNumber)
    depositTKN(tknAmount, tokenName, user)
    (
        simulatorMasterVaultTknBalance, 
        simulatorStakingLedgerTknBalance, 
        simulatorErc20ContractTknBalance, 
        simulatorBntknAmount) = getSimulationDepositTknOutputs(tokenName)
    depositTknComparisson(
        blockNumber, 
        txHash, 
        tokenName,
        tknAmount, 
        contractMasterVaultTknBalance, simulatorMasterVaultTknBalance,
        contractStakingLedgerTknBalance, simulatorStakingLedgerTknBalance,
        contractErc20ContractTknBalance, simulatorErc20ContractTknBalance,
        contractBntknAmount, simulatorBntknAmount)
    return(None)

def contractWithdrawalInitiated(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the depositTKN event from the contracts.
    """
    tokenName = getContractTokenName(df, index)
    tokenDecimals = getContractTokenDecimals(df, index)
    withdrawValue = getContractReserveTokenAmount(df, index, tokenDecimals)
    contractPoolTokenAmount = getContractWithdrawalInitiatedOutputs(df, index)
    newTimestamp(blockNumber)
    beginCooldown(withdrawValue, tokenName, user)
    simulatorPoolTokenAmount = getSimulatedWithdrawalInitiatedOutputs()
    withdrawalInitiatedComparisson(
        blockNumber, 
        txHash, 
        tokenName, 
        withdrawValue, 
        contractPoolTokenAmount, 
        simulatorPoolTokenAmount)
    return(None)

def contractDepositBnt(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the depositTKN event from the contracts.
    """
    bntAmount = getContractBntAmount(df, index)
    contractPoolTokenAmount, contractVbntAmount = getContractDepositBntOutputs(df, index)
    newTimestamp(blockNumber)
    depositBNT(bntAmount, user)
    simulatorPoolTokenAmount, simulatorVbntAmount = getSimulatorDepositBntOutputs()
    depositBntComparisson(
        blockNumber, 
        txHash, 
        bntAmount, 
        contractPoolTokenAmount, 
        contractVbntAmount, 
        simulatorPoolTokenAmount, 
        simulatorVbntAmount)
    return(None)

def contractTrade(df, index, blockNumber, txHash):
    """
    @Dev
    Processes the trade event from the contracts.
    """
    (
        sourceToken, 
        sourceTokenAmount, 
        targetToken, 
        contractTargetTokenAmount, 
        contractBntPrevLiquidity, 
        contractBntNewLiquidity, 
        contractTknPrevLiquidity, 
        contractTknNewLiquidity, 
        contractFeeAmount, 
        impliedTradingFee, 
        impliedNetworkFee, 
        impliedFeeToStakingLedger, 
        impliedBntToVortexLedger) = getContractTradeOutputs(df, index, txHash)
    trade(sourceTokenAmount, sourceToken, targetToken, user)
    (
        trueTradingFee, 
        simulatedBntPrevLiquidity, 
        simulatedBntNewLiquidity, 
        simulatedTknPrevLiquidity, 
        simulatedTknNewLiquidity) = getSimulatedTradingPoolVars(sourceToken, targetToken)
    (
        simulatedTargetTokenAmount, 
        simulatedFeeToStakingLedger, 
        simulatedBntToVortexLedger) = getSimulatedTradingOutputs(targetToken)
    trueNetworkFee = nonUsers["globalProtocolSettings"]["networkFee"][-1]
    simulatedFeeAmount = getSimulatedFeeAmount(
        sourceToken, 
        sourceTokenAmount, 
        simulatedBntPrevLiquidity, 
        simulatedTknPrevLiquidity, 
        trueTradingFee)
    tradeComparisson(
    blockNumber, 
    txHash, 
    sourceTokenAmount, 
    sourceToken, 
    targetToken, 
    contractTargetTokenAmount, simulatedTargetTokenAmount, 
    contractBntPrevLiquidity, simulatedBntPrevLiquidity, 
    contractTknPrevLiquidity, simulatedTknPrevLiquidity, 
    contractBntNewLiquidity, simulatedBntNewLiquidity, 
    contractTknNewLiquidity, simulatedTknNewLiquidity, 
    impliedTradingFee, trueTradingFee, 
    impliedNetworkFee, trueNetworkFee, 
    contractFeeAmount, simulatedFeeAmount, 
    impliedFeeToStakingLedger, simulatedFeeToStakingLedger, 
    impliedBntToVortexLedger, simulatedBntToVortexLedger)
    return(None)

contractFunctionDictionary = {
    "NetworkFeePPMUpdated" : contractNetworkFeePPMUpdated,
    "TradingFeePPMUpdated" : contractTradingFeePPMUpdated,
    "DepositingEnabled" : contractDepositingEnabled,
    "TradingEnabled" : contractTradingEnabled,
    "FundingLimits" : contractFundingLimits,
    "depositTKN" :  contractDepositTkn,
    "WithdrawalInitiated" : contractWithdrawalInitiated,
    "depositBNT" : contractDepositBnt,
    "trade" : contractTrade
}

def securityMonitor(events):
    df = pd.read_csv(events)
    for index in df.index:
        print(index)
        copyRows()
        blockNumber = getContractBlockNumber(df, index)
        newTimestamp(blockNumber)
        txHash = getTxHash(df, index)
        type = df["type"][index]
        function = contractFunctionDictionary[type]
        function(df, index, blockNumber, txHash) 

def monitorCharts(monitor):
    monitorTokenDeposits = pd.DataFrame.from_dict(monitor["token deposits"])
    monitorWithdrawalsInitiated = pd.DataFrame.from_dict(monitor["withdrawals initiated"])
    monitorBntDeposits = pd.DataFrame.from_dict(monitor["bnt deposits"])
    monitorTrades = pd.DataFrame.from_dict(monitor["trades"])
    sns.set(rc={
        'axes.facecolor':'black', 
        'figure.facecolor':'black', 
        'text.color': 'white',
        'xtick.color': 'white', 
        'ytick.color': 'white', 
        'axes.grid': False, 
        'axes.labelcolor': 'white'})
    monitorTokenDeposits.columns
    for monitordict in [monitorTokenDeposits, monitorWithdrawalsInitiated, monitorBntDeposits, monitorTrades]:
        begin = [i for i,c in enumerate(monitordict.keys()) if 'simulated' in c][0]-1
        zippy = list(zip(monitordict.keys()[begin::2],monitordict.keys()[begin+1::2]))
        count = 0
        for i in range(len(zippy)):
            monitordict.loc[:,zippy[i]] = monitordict.loc[:,zippy[i]].astype(float)
            fig = plt.figure()
            fig.set_size_inches(6,6)
            print(zippy[i][0])
            print("Compare max deviation from ethereum vs simulated:")
            print(abs((monitordict.loc[:,zippy[i][1]] - monitordict.loc[:,zippy[i][0]])/monitordict.loc[:,zippy[i][1]]).max())
            sns.lineplot(data = monitordict, x = 'block number', y = (monitordict.loc[:,zippy[i][1]] - monitordict.loc[:,zippy[i][0]])/monitordict.loc[:,zippy[i][1]], color='r')
            plt.show()
            count += 1
    return(None)

# Playground

In [84]:
# whitelistedTokens = ["bnt", "vbnt", "eth", "wbtc", "link"]    # ["bat", "bnt", "vbnt", "dai", "enj", "link", "mana", "matic", "mkr", "mln", "ocean", "omg", "ren", "rsr", "usdc", "wbtc", "eth"]
# users = ["alice", "bob", "charlie"] # "carol", "carlos", "chuck", "chad", "craig", "dan", "dave", "erin", "eve", "yves", "faythe", "frank", "grace", "heidi", "ivan", "judy", "mallory", "mallet", "darth", "mike", "niaj"
# priceFeeds = pd.read_csv("BatBntDaiEnjLinkManaMaticMkrMlnOceanOmgRenRsrUsdcUsdtWbtcEth.csv")
# timestamp = 0
# users, nonUsers, pendingWithdrawals, factions = systemBuilder(timestamp, whitelistedTokens, users)

# Perform a Monte Carlo Simulation

In [85]:
# whitelistedTokens = ["bat", "bnt", "vbnt", "dai", "enj", "link", "mana", "matic", "mkr", "mln", "ocean", "omg", "ren", "rsr", "usdc", "wbtc", "eth"]
# users = ["alice", "bob", "charlie"] # "carol", "carlos", "chuck", "chad", "craig", "dan", "dave", "erin", "eve", "yves", "faythe", "frank", "grace", "heidi", "ivan", "judy", "mallory", "mallet", "darth", "mike", "niaj"
# priceFeeds = pd.read_csv("BatBntDaiEnjLinkManaMaticMkrMlnOceanOmgRenRsrUsdcUsdtWbtcEth.csv")
# timestamp = 0
# users, nonUsers, pendingWithdrawals, factions = systemBuilder(timestamp, whitelistedTokens, users)
# monteCarloSimulation()

# Run the Security Monitor

In [86]:
# timestamp = 14609474
# users = ["superuser"]
# user = "superuser"
# whitelistedTokens = ["bnt", "vbnt", "dai", "eth", "link"]
# monitor = getEmptyMonitor()
# contractTokenDictionary = getContracttokenDictionary()
# events = "security_monitor_trial_1.csv"
# users, nonUsers, pendingWithdrawals, factions = systemBuilder(timestamp, whitelistedTokens, users)
# securityMonitor(events)
# monitorCharts(monitor)

# Recreate a Fork of Mainnet/Tenderly (system only, lone superuser)

In [87]:
# tokenWhitelist = "whitelist.csv"
# poolStats = "poolstats35.csv"
# globalSettings = "global35.csv"
# timestamp, users, nonUsers, pendingWithdrawals, factions, whitelistedTokens = setPools(tokenWhitelist, poolStats, globalSettings)

In [88]:
# users["superuser"]["ethBalance"][-1] = Decimal("270529.756799871182300702")
# users["superuser"]["bntBalance"][-1] = Decimal("32140.574329517846156036")
# users["superuser"]["daiBalance"][-1] = Decimal("1024184.030198849852054043")
# users["superuser"]["linkBalance"][-1] = Decimal("28107454.02890364772346443")
# users["superuser"]["vbntBalance"][-1] = Decimal("0")
# users["superuser"]["bnlinkBalance"][-1] = Decimal("6903790.792890223036796312")
# users["superuser"]["bnbntBalance"][-1] = Decimal("389.970562707468556262")
# users["superuser"]["bnethBalance"][-1] = Decimal("1101.871672104927003383")
# users["superuser"]["bndaiBalance"][-1] = Decimal("0")
# users["superuser"]["bnvbntBalance"][-1] = Decimal("0")
#
# doNothing()

In [89]:
# copyRows()
# newTimestamp()
# forceMovingAverage("dai", "superuser")
#
# copyRows()
# newTimestamp()
# forceMovingAverage("eth", "superuser")
#
# copyRows()
# newTimestamp()
# depositTKN("9999", "dai", "superuser")
#
# copyRows()
# newTimestamp()
# beginCooldown("9999", "dai", "superuser")

In [90]:
# def getDeficitSurplus(tokenName):
#     """
#     @Dev
#     Returns the state of an asset inside the system (deficit or surplus) as a percentage. 
#     """
#     masterVaultBalance = nonUsers["masterVault"][f"{tokenName}Balance"][-1]
#     stakingLedgerBalance = nonUsers["stakingLedger"][f"{tokenName}Balance"][-1]
#     state = "surplus" if masterVaultBalance > stakingLedgerBalance else "deficit"
#     proportion = (abs(masterVaultBalance - stakingLedgerBalance)/stakingLedgerBalance)*100
#     print(f"Checking the deficit or surplus of {tokenName}")
#     print(f"masterVaultBalance of {tokenName} is {masterVaultBalance:.18f} and stakingLedgerBalance of {tokenName} is {stakingLedgerBalance:.18f}")
#     print(f"{tokenName} is in {state} by {proportion:.18f}%")
#     return(state, proportion)


# tokens = (tokenName for tokenName in whitelistedTokens if tokenName != "bnt" and tokenName != "vbnt")
# for tokenName in tokens:
#     getDeficitSurplus(tokenName)


# Barak's JSON parser

In [91]:
Decimal("10100458.465138707919646012")/Decimal("1612978.311243796501599847")





Decimal('6.2619927340188866296842265576156750580174597711601104562876575273630426816506622776800453635540606643437407724073531644105598148611399499924616196872796420')

In [92]:
Decimal("20199973.729617847159099597")/Decimal("3225805.999403275794537015")


Decimal('6.2619927340188870000000007768661754182582124521834420719263785474724679614770378551908876786260965304974961533034162465885804788219864490395216368595256375')

In [93]:
# print(nonUsers["linkPool"]["emaRate"][-1])

In [94]:
# jsonFile = "BancorNetworkReduceTradingLiquidity.json"

# def makeValidationSystem(users, ):
    
    
#     timestamp = 0
#     users, nonUsers, pendingWithdrawals, factions = systemBuilder(timestamp, whitelistedTokens, users)
#     return(users, nonUsers, pendingWithdrawals, factions)

# def validateJsonFile(jsonFile):
#     whitelistedTokens = ["tkn", "vbnt"]
#     users = ["tknProvider"] 

#     print(f'checking {jsonFile}...')
#     with open(jsonFile, 'r') as read_file:
#         data = json.load(read_file)
#         print(data['tradingFee'])
        # for dic in data:
        #     errors = []
        #     a, b, c, e, m, n, x, p, q, r, s = Decimal(dic['a']), Decimal(dic['b']), Decimal(dic['c']), Decimal(dic['e']), Decimal(dic['m']), Decimal(dic['n']), Decimal(dic['x']), Decimal(dic['p']), Decimal(dic['q']), Decimal(dic['r']), Decimal(dic['s'])
        #     w = Decimal('0')
        #     m /= Decimal('1000000')
        #     n /= Decimal('1000000')
        #     mp, mq, mr, ms, mt, mu = Withdrawal(a, b, c, e, m, n, w, x)
        #     mp = Decimal(f'{mp:.12f}')
        #     mq = Decimal(f'{mq:.12f}')
        #     mr = Decimal(f'{mr:.12f}')
        #     ms = Decimal(f'{ms:.12f}')
        #     if mp == p and mq == q and mr == r and ms == s:
        #         pass
        #     else:
        #         errors.append({
        #             'a' : str(a), 
        #             'b' : str(b), 
        #             'c' : str(c), 
        #             'e' : str(e),
        #             'm' : str(m),
        #             'n' : str(n), 
        #             'x' : str(x), 
        #             ' p' : str(p),
        #             'mp' : str(mp),
        #             ' q' : str(q),
        #             'mq' : str(mq),
        #             ' r' : str(r),
        #             'mr' : str(mr),
        #             ' s' : str(s),
        #             'ms' : str(ms), 
        #         })
        #     with open(f'{jsonFile}_ERRORS.json', 'w') as outfile:
        #         json.dump(errors, outfile, indent=4)
        # print(f'There were {len(errors)} errors')
        # print (f'{jsonFile} done')

In [95]:
# jsonFile = "BancorNetworkReduceTradingLiquidity.json"

# validateJsonFile(jsonFile)

In [96]:
simulation = []
validation = []
indx = 0
DECIMALS: int = 18
QDECIMALS = Decimal(10) ** -DECIMALS

In [97]:
def validationEnvironment(data, users, nonUsers, pendingWithdrawals, factions):

    timestamp = 0
    for user in data["users"]:
        users[user["id"]]["tknBalance"][-1] = Decimal(user["tknBalance"])
        users[user["id"]]["bntBalance"][-1] = Decimal(user["bntBalance"])
    nonUsers["tknPool"]["tradingFee"][-1] = Decimal(data["tradingFee"].replace("%", ""))/Decimal("100")
    nonUsers["tknPool"]["externalProtectionVaultBalance"][-1] = Decimal(data["epVaultBalance"])
    nonUsers["tknPool"]["bntFundingLimit"][-1] = Decimal(data["bntFundingLimit"])
    nonUsers["globalProtocolSettings"]["networkFee"][-1] = Decimal(data["networkFee"].replace("%", ""))/Decimal("100")
    nonUsers["globalProtocolSettings"]["withdrawalFee"][-1] = Decimal(data["withdrawalFee"].replace("%", ""))/Decimal("100")
    nonUsers["globalProtocolSettings"]["bntMinLiquidity"][-1] = Decimal(data["bntMinLiquidity"])
    return timestamp, users, nonUsers, pendingWithdrawals, factions, whitelistedTokens

def convertWithdrawal(tokenName, amount):
    stakedBalance = nonUsers["stakingLedger"][f"{tokenName}Balance"][-1]
    poolTokenSupply = nonUsers["erc20Contracts"][f"bn{tokenName}Balance"][-1]
    poolTokenRate = getBntknRate(stakedBalance, poolTokenSupply)
    withdrawValue = amount/poolTokenRate
    return(withdrawValue)

def performOperation(operation, indx, users, nonUsers):

    transaction_type = operation['type']
    user_name = user = operation['userId']
    mined = operation['mined']
    amt = operation['amount']
    if type(amt) == dict:
        amt = Decimal(0)
    else:
        amt = Decimal(amt)

    if operation["mined"] == True:
        newTimestamp()
    action = operation["type"]
    if action == "depositBNT":
        depositBNT(operation["amount"], operation["userId"])
    elif action == "depositTKN":
        depositTKN(operation["amount"], "tkn", operation["userId"])
    elif action == "enableTrading":
        enableTrading("tkn", operation["amount"]["bntVirtualBalance"], operation["amount"]["baseTokenVirtualBalance"], nonUsers["tknPool"]["bntFundingLimit"][-1])
    elif action == "tradeBNT":
        trade(operation["amount"], "bnt", "tkn", operation["userId"])
    elif action == "tradeTKN":
        trade(operation["amount"], "tkn", "bnt", operation["userId"])
    elif action == "withdrawBNT":
        withdrawValue = convertWithdrawal("bnt", Decimal(operation["amount"]))
        beginCooldown(withdrawValue, "bnt", operation["userId"])
        withdraw(operation["userId"], str(Decimal(pendingWithdrawals["idCounter"]) - Decimal("1")))
    elif action == "withdrawTKN":
        withdrawValue = convertWithdrawal("tkn", Decimal(operation["amount"]))
        beginCooldown(withdrawValue, "tkn", operation["userId"])
        withdraw(operation["userId"], str(Decimal(pendingWithdrawals["idCounter"]) - Decimal("1")))
    elif action == "reduceTradingLiquidity":
        reduceTradingLiquidity("tkn", operation["amount"])
    elif action == "setFundingLimit":
        changeBntFundingLimit("tkn", operation["amount"])

    tokenName = 'tkn'
    expected = operation['expected']

    # logging
    collect = {
          "indx": [indx],
          "action": [transaction_type],
          "amount": [amt],
          "tkn_dayTrader": [Decimal(expected['tknBalances']['dayTrader']).quantize(QDECIMALS)],
          "tkn_tknProvider": [Decimal(expected['tknBalances']['tknProvider']).quantize(QDECIMALS)],
          "tkn_bntProvider": [Decimal(expected['tknBalances']['bntProvider']).quantize(QDECIMALS)],
          "tkn_masterVault": [Decimal(expected['tknBalances']['masterVault']).quantize(QDECIMALS)],
          "tkn_epVault": [Decimal(expected['tknBalances']['epVault']).quantize(QDECIMALS)],
          "bnt_dayTrader": [Decimal(expected['bntBalances']['dayTrader']).quantize(QDECIMALS)],
          "bnt_tknProvider": [Decimal(expected['bntBalances']['tknProvider']).quantize(QDECIMALS)],
          "bnt_bntProvider": [Decimal(expected['bntBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnt_masterVault": [Decimal(expected['bntBalances']['masterVault']).quantize(QDECIMALS)],
          "bntkn_dayTrader": [Decimal(expected['bntknBalances']['dayTrader']).quantize(QDECIMALS)],
          "bntkn_tknProvider": [Decimal(expected['bntknBalances']['tknProvider']).quantize(QDECIMALS)],
          "bntkn_bntProvider": [Decimal(expected['bntknBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnbnt_dayTrader": [Decimal(expected['bnbntBalances']['dayTrader']).quantize(QDECIMALS)],
          "bnbnt_tknProvider": [Decimal(expected['bnbntBalances']['tknProvider']).quantize(QDECIMALS)],
          "bnbnt_bntProvider": [Decimal(expected['bnbntBalances']['bntProvider']).quantize(QDECIMALS)],
          "bnbnt_bntPool": [Decimal(expected['bnbntBalances']['bntPool']).quantize(QDECIMALS)],
          "bntCurrentPoolFunding": [Decimal(expected['bntCurrentPoolFunding']).quantize(QDECIMALS)],
          "tknStakedBalance": [Decimal(expected['tknStakedBalance']).quantize(QDECIMALS)],
          "bntStakedBalance": [Decimal(expected['bntStakedBalance']).quantize(QDECIMALS)],
          "tknTradingLiquidity": [Decimal(expected['tknTradingLiquidity']).quantize(QDECIMALS)],
          "bntTradingLiquidity": [Decimal(expected['bntTradingLiquidity']).quantize(QDECIMALS)],
    }
    validation.append(pd.DataFrame(collect))

    collect = {
          "indx": [indx],
          "action": [transaction_type],
          "amount": [amt],
          "tkn_dayTrader": [users['dayTrader'][f"{tokenName}Balance"][-1].quantize(QDECIMALS)],
          "tkn_tknProvider": [users['tknProvider'][f"{tokenName}Balance"][-1].quantize(QDECIMALS)],
          "tkn_bntProvider": [users['bntProvider'][f"{tokenName}Balance"][-1].quantize(QDECIMALS)],
          "tkn_masterVault": [nonUsers["masterVault"][f"{tokenName}Balance"][-1].quantize(QDECIMALS)],
          "tkn_epVault": [nonUsers["tknPool"]["externalProtectionVaultBalance"][-1].quantize(QDECIMALS)],
          "bnt_dayTrader": [users['dayTrader'][f"bntBalance"][-1].quantize(QDECIMALS)],
          "bnt_tknProvider": [users['tknProvider'][f"bntBalance"][-1].quantize(QDECIMALS)],
          "bnt_bntProvider": [users['bntProvider'][f"bntBalance"][-1].quantize(QDECIMALS)],
          "bnt_masterVault": [nonUsers["masterVault"][f"bntBalance"][-1].quantize(QDECIMALS)],
          "bntkn_dayTrader": [users['dayTrader'][f"bntknBalance"][-1].quantize(QDECIMALS)],
          "bntkn_tknProvider": [users['tknProvider'][f"bntknBalance"][-1].quantize(QDECIMALS)],
          "bntkn_bntProvider": [users['bntProvider'][f"bntknBalance"][-1].quantize(QDECIMALS)],
          "bnbnt_dayTrader": [users['dayTrader'][f"bnbntBalance"][-1].quantize(QDECIMALS)],
          "bnbnt_tknProvider": [users['tknProvider'][f"bnbntBalance"][-1].quantize(QDECIMALS)],
          "bnbnt_bntProvider": [users['bntProvider'][f"bnbntBalance"][-1].quantize(QDECIMALS)],
          "bnbnt_bntPool": [nonUsers["protocolWallet"]["bnbntBalance"][-1].quantize(QDECIMALS)],
          "bntCurrentPoolFunding": [nonUsers[f"{tokenName}Pool"]["bntFundingAmount"][-1].quantize(QDECIMALS)],
          "tknStakedBalance": [nonUsers["stakingLedger"]["tknBalance"][-1].quantize(QDECIMALS)],
          "bntStakedBalance": [nonUsers["stakingLedger"]["bntBalance"][-1].quantize(QDECIMALS)],
          "tknTradingLiquidity": [nonUsers[f"{tokenName}Pool"][f"{tokenName}TradingLiquidity"][-1].quantize(QDECIMALS)],
          "bntTradingLiquidity": [nonUsers[f"{tokenName}Pool"]["bntTradingLiquidity"][-1].quantize(QDECIMALS)],
    }
    simulation.append(pd.DataFrame(collect))
    return(None)

import time
def validateJsonfile(operations, timestamp, users, nonUsers, indx = 0):

    for operation in operations:
        indx += 1
        # if indx % 500 == 0:
        #     time.sleep(1)
        performOperation(operation, indx, users, nonUsers)
    return(None)













In [108]:
import json
# jsonFile1 = "BancorNetworkReduceTradingLiquidity.json"
jsonFile2 = "BancorNetworkComplexFinancialScenario1 (1).json"
jsonf = '/Users/mikewcasale/Local/projects/bancor/simulator-v3/bancor3_simulator/tests/data/scenarios_2.json'
# jsonf = 'scenarios_2.json'

with open(jsonf, "r") as f:
    data = json.load(f)
operations = data['operations']
whitelistedTokens = ["tkn", "bnt", "vbnt"]
users = [user['id'] for user in data["users"]]
timestamp = 0
users, nonUsers, pendingWithdrawals, factions = systemBuilder(timestamp, whitelistedTokens, users)
timestamp, users, nonUsers, pendingWithdrawals, factions, whitelistedTokens =  validationEnvironment(data, users, nonUsers, pendingWithdrawals, factions)



In [109]:
validateJsonfile(operations, timestamp, users, nonUsers)

Depositing 30000000 tkn
stakedTkn is 0.000000000000000000 and bntknSupply is 0.000000000000000000
bntknrate is 1.000000000000000000
** bntknRate 1
bntknRate is 1.000000000000000000 and tknAmount is 30000000.000000000000000000
bntknAmount is 30000000.000000000000000000
** bntknAmount 30000000
Adjusting the pool depth.
tradingEnabled = False
spotRate is 0.000000000000000000, emaRate is 0.000000000000000000
bntTradingLiquidity is 0.000000000000000000, masterVaultTknBalance is 30000000.000000000000000000
bntRemainingFunding is 0.000000000000000000
bntIncrease is 0.000000000000000000, tknIncrease is 0.000000000000000000
Handling the protocol-owned liquidity.
stakedBnt is 0.000000000000000000 and bnbntSupply is 0.000000000000000000
bnbntRate is 1.000000000000000000
bnbntRate is 1.000000000000000000 and bntAmount is 0.000000000000000000
bnbntAmount is 0.000000000000000000
updatedBntLiquidity is 0.000000000000000000 and updatedTknLiquidity is 0.000000000000000000
spotRate is 0.0000000000000000

/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_97134/2444798821.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation.append(pd.DataFrame(collect))
/var/folders/rt/qnj8r6yd6131ccxkw_k9d9gc0000gn/T/ipykernel_97134/2444798821.py:117: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulation.append(pd.DataFrame(collect))


Added a new timestamp; the last blocktime was 2 seconds
Depositing 88276 bnt
stakedBnt is 10000000.000000000000000000 and bnbntSupply is 9999996.156707742187958060
bnbntRate is 0.999999615670774219
bnbntRate is 0.999999615670774219 and bntAmount is 88276.000000000000000000
bnbntAmount is 88275.966072953264938419


***TRADING LIQUIDITIES***

TKN POOL
bntTradingLiquidity : 10004862.247508554070448866
tknTradingLiquidity : 30016317.455011494152348096

VBNT POOL
bntTradingLiquidity : 0.000000000000000000
vbntTradingLiquidity : 0.000000000000000000



***USER WALLETS***

DAYTRADER
tknBalance : 24536.090840686095086963

bntknBalance : 0.000000000000000000

bntBalance : 5132.703738525473911296

bnbntBalance : 0.000000000000000000

vbntBalance : 0.000000000000000000

bnvbntBalance : 0.000000000000000000

TKNPROVIDER
tknBalance : 68679012.000000000000000000

bntknBalance : 31320987.661292660036303398

bntBalance : 0.000000000000000000

bnbntBalance : 0.000000000000000000

vbntBalance : 0.000000

In [100]:
simulation, validation = pd.concat(simulation), pd.concat(validation)



In [101]:
simulation = simulation.drop('indx', axis=1).reset_index().drop('index', axis=1)

# .replace(0,Decimal(0.0000000000000000001) allows for Decimal division of approx. zero values
simulation = simulation.drop('action', axis=1).replace(0,Decimal(0.0000000000000000001))
simulation



,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,bnt_masterVault,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,30000000,10000.000000000000000000,70000000.000000000000000000,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,9.99999999999999975245926835260131855729159055...,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...
1,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,70000000.000000000000000000,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,2000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,2000000.000000000000000000,2000000.000000000000000000,30000000.000000000000000000,2000000.000000000000000000,6000000.000000000000000000,2000000.000000000000000000
2,1000000,10000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,4000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,4000000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
3,100000,10000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,9900000.000000000000000000,4000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,100000.000000000000000000,3900000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
4,2000,8000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31002000.000000000000000000,9.99999999999999975245926835260131855729159055...,10664.889185135810698217,9.99999999999999975245926835260131855729159055...,9900000.000000000000000000,3999335.110814864189301783,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000000.000000000000000000,4000001.333111148141976337,12002000.000000000000000000,3999334.777537077153807699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,36883,17849

In [110]:
simulation.head(50)

,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,bnt_masterVault,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,30000000,10000.000000000000000000,70000000.000000000000000000,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,9.99999999999999975245926835260131855729159055...,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...
1,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,70000000.000000000000000000,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,2000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,2000000.000000000000000000,2000000.000000000000000000,30000000.000000000000000000,2000000.000000000000000000,6000000.000000000000000000,2000000.000000000000000000
2,1000000,10000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,4000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,4000000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
3,100000,10000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,9900000.000000000000000000,4000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,100000.000000000000000000,3900000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
4,2000,8000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31002000.000000000000000000,9.99999999999999975245926835260131855729159055...,10664.889185135810698217,9.99999999999999975245926835260131855729159055...,9900000.000000000000000000,3999335.110814864189301783,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000000.000000000000000000,4000001.333111148141976337,12002000.000000000000000000,3999334.777537077153807699
5,5332,23940.072142346676806238,69000000.000000000000000000,9.99999999999999975245926835260131855729159

In [111]:
simulation.to_csv('mark_results.csv', index=False)

In [103]:
validation = validation.drop('indx', axis=1).reset_index().drop('index', axis=1)

# .replace(0,Decimal(0.0000000000000000001) allows for Decimal division of approx. zero values
validation = validation.drop('action', axis=1).replace(0,Decimal(0.0000000000000000001))
validation

,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,bnt_masterVault,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,30000000,10000.000000000000000000,70000000.000000000000000000,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,9.99999999999999975245926835260131855729159055...,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...
1,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,70000000.000000000000000000,9.99999999999999975245926835260131855729159055...,30000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,2000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,2000000.000000000000000000,2000000.000000000000000000,30000000.000000000000000000,2000000.000000000000000000,6000000.000000000000000000,2000000.000000000000000000
2,1000000,10000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000000.000000000000000000,4000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,4000000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
3,100000,10000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31000000.000000000000000000,9.99999999999999975245926835260131855729159055...,10000.000000000000000000,9.99999999999999975245926835260131855729159055...,9900000.000000000000000000,4000000.000000000000000000,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,100000.000000000000000000,3900000.000000000000000000,4000000.000000000000000000,31000000.000000000000000000,4000000.000000000000000000,12000000.000000000000000000,4000000.000000000000000000
4,2000,8000.000000000000000000,69000000.000000000000000000,9.99999999999999975245926835260131855729159055...,31002000.000000000000000000,9.99999999999999975245926835260131855729159055...,10664.889185135810698217,9.99999999999999975245926835260131855729159055...,9900000.000000000000000000,3999335.110814864189301783,...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,9.99999999999999975245926835260131855729159055...,100000.000000000000000000,3900000.000000000000000000,4000001.333111148141976337,31000000.000000000000000000,4000001.333111148141976337,12002000.000000000000000000,3999334.777537077153807699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,36883,17843

# Final comparison results

In [104]:
result_df = (1 - (simulation / validation)) * 100
result_df

# trade_bnt_for_tkn

,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,bnt_masterVault,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,0,-0.0335340500506767860902650109020984916158129...,-3.1286443503118589938427012405210799930992233...,0,0.00001662711881149920433029518657988464941001...,0,0.02911182676731177247190860394469704267708615...,0,1.15391823732668505980336296364123916287567752...,-0.0264790389713599380140535992866403098706224...,...,0,0,0,-9.6610720118847650457763834519217053126370287...,-0.0320222859980961563738549779247088928445042...,-0.0264645434745051367764552459636866096090365...,-3.2775769056825972616864537672152222844541446...,-0.0264645434745051367764552459636866096090365...,-0.0677976171917025083919800023835753508967450...,-0.0264790807995511826189100
395,0,-0.0335340500506767860902650109020984916158129...,-3.1286443503118589938427012405210799930992233...,0,0.00001662711881149920433029518657988464941001...,0,0.02911182676731177247190860394469704267708615...,0,1.15437992235558240413416590158323621197534457...,-0.0264790389713599380140535992866403098706224...,...,0,0,0,-9.6636719111121418666750059042502517540434531...,-0.0320350964679352827893025616457862952732930...,-0.0264645434745051367764552459636866096090365...,-3.2775769056825972616864537672152222844541446...,-0.0264645434745051367764552459636866096090365...,-0.0677976171917025083919800023835753508967450...,-0.0264790807995511826189100
396,0,-0.0335340500506767860902650109020984916158129...,-3.1286443503118589938427012405210799930992233...,0,0.00001662711881149920433029518657988464941001...,0,0.02911182676731177247190860394469704267708615...,0,1.23662885558504573376634071967286554804498223...,-0.0264790389713599380140535992866403098706224...,...,0,0,0,-1.0066312666389674397965079762979423280271736...,-0.0317676374094300966643865645065683480407334...,-0.0264645434745051367764552459636866096090365...,-3.2775769056825972616864537672152222844541446...,-0.0264645434745051367764552459636866096090365...,-0.0677976171917025083919800023835753508967450...,-0.0264790807995511826189100
397,0,-0.0335807065810962831552942915885322852258937...,-3.1286443503118589938427012405210799930992233...,0,0.00001675083477073038250212908581511149659642...,0,0.02925568906554716391742810635032673275405637...,0,1.23662885558504573376634071967286554804498223...,-0.0264789436472749292207263450370659225941369...,...,0,0,0,-1.0066312666389674397965079762979423280271736...,-0.0317676374094300966643865645065683480407334...,-0.0264645434745051367764552459636866096090365...,-3.3023698623400541513988176407645086430510196...,-0.0264645434745051367764552459636866096090365...,-0.0677977123170163427202273792545160268697187...,-0.0264789855228658165081641219469637251158549...


In [105]:
result_df.head(50)

,amount,tkn_dayTrader,tkn_tknProvider,tkn_bntProvider,tkn_masterVault,tkn_epVault,bnt_dayTrader,bnt_tknProvider,bnt_bntProvider,bnt_masterVault,...,bntkn_bntProvider,bnbnt_dayTrader,bnbnt_tknProvider,bnbnt_bntProvider,bnbnt_bntPool,bntCurrentPoolFunding,tknStakedBalance,bntStakedBalance,tknTradingLiquidity,bntTradingLiquidity
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,-3.2258031259093188453738947519316507160225601...,0,0,2.49708832759388096552417582942530880062612067...
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,6.96572931962595264819269177460560625080881689...,-2.5930650969171001003130858432755067288709396...,0,-3.2258031259093188453738947519316507160225601...,0,0,2.49708832759388096552417582942530880062612067...
7,0,0,0,0,0,0,0,0,0,1.24854369569825012088758360344504285901951921...,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.27208615252600887916246696681839019931339855...,1.24927144970375688885297745466156008777604513...,-3.2029895977140322525401679682396090889067366...,1.24927144970375688885297745466156008777604513...,-3.6960688838514903130371914566431652607837816...,3.74563249139082144828626374413796320093918101...
8,0,-1.2206286170952129219343096029494086326976683...,0,0,9.61346731413218814042059064113538096879412377...,0,0,0,0,1.24851049283427700009358540552147546748540156...,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.27208615252600887916246696681839019931339855...,1.24927144970375688885297745466156008777604513...,0,1.24927144970375688885297745466156008777604513...,-3.6960836983900613563640504108371290187437767...,4.99404391003805874810012966034754394736629612...
9,0,-1.2518516903118937631112424710774717027340311...,0,0,9.61327847552137839326165670090973686770604335...,0,0,0,0,1.24854226381611268308476276329546671338661944...,...,0,0,0,6.96572931962595264819269177460560625080881689...,1.27208615252600887916246696681839019931339855...,1.24927138592649198036367491661817726956795367...,0,1.24927138592649198036367491661817726956795367...,-3.6959894103844067187159543184433092221731991...,4.99417105776695941906672142843326812019128593...


In [106]:
# result_df.to_csv('mark_results.csv', index=False)

In [107]:
# changed_bnt_trading_liquidity

NameError: name 'changed_bnt_trading_liquidity' is not defined